# Imports

In [1]:
import math, pyperclip, os, re, decimal
from decimal import Decimal
import numpy as np
from functools import reduce
from itertools import product
from datetime import datetime, date, time

# Docs

In [2]:
"""
Data types: Real, Complex, Int, Boolean, String, Undefined
Data structures: Scalar, Vector, Matrix, ...
Variables: v
Functions: fun
Named operators: v operation u
Conversion specifier: 45deg
Data properties: object.property
Oppertation: Numeric, Sets, Comparison 

Data Structures
Tensor: [a, b,, c, d,,, ...]
List: (a, b, c, ...)
Function Body: {expression}

Implicit Operations
2(expression)      *
(expression)2      *
var(expression)    *
fun(parameters)    call

Named operators
on two values: v operation u
on one value: v.operation

Input and Output conversion
1m + 12cm @ cm
2hr + 45min + 1hr + 30min @ datetime
0b1100 * 0xFF @ dec

Basic Operations:
Standard:       | Bitwise: int&bool | Comparison:           | 
+           add | ~             not | ==             equals | 
-      subtract | &&            and | !=          not equal | 
*      multiply | ||             or | <           less than | 
/        divide | <xor>         xor | >        greater then | 
//      int div | <<     left shift | <=      less or equal | 
%       modulus | >>    right shift | >=   greater or equal | 
^         power |                   |                       | 
!     factorial |                   |                       | 
|val|  absolute |                   |                       | 
=    assignment |                   |                       | 

Ternary Operators
= =                return values = function = body
a < x < b          between
a if cond else b   

Higher Ranking Data Structure Operations:
Vector:                 | Matrix:                          | Reductions:
<dot>       dot product | <matmul>   matrix multiplication | <all>
<cross>   cross product | |mat|                            | <any>
|vec|            length | .T              transpose matrix | <
.length          length | 
.angle            angle | 



1 + sqrt 4
1 + sqrt x
1 + $4
1 + 3 root 8
1 + b root x
1 + b $ x

10 nPr 2
n nPr r
10 nCr 2
n nCr r

[1, 2,, 3, 4] # [5, 6,, 7, 8]
[1, 2,, 3, 4].T
mat1.T

5*x

sin 30deg
sin pi
sin2 pi
sini 0.5

Function definitions
fun = x => x^2               single value output
fun = x => [x, x*2, x^2]     tensor outputs
fun = x => (x, x*2, x^2)     multiple outputs
fun = x => {                 piecewise function
    0,   if x < 0 ; 
    x^2, if 0 <= x <= 1 ; 
    x,   else
}
fun(x) = {                  piecewise function
    0   if x < 0
    x^2 if 0 <= x <= 1
    x   else
}

fun(x,y,z) = {
    a = x+2
    b = y*2
    c = z^2
    (a,b,c)
}

if x < 0 {
    x = 0
}

for i = 0:10{
    x += i
}

while x < 0 {
    x += 1
}

5(m/s^2) / 3(m/s)
8(L/100km)

Statistical Operations:
"""
None

In [3]:
"""
Conversion specifiers

Time:
    Seconds: s, ms, µs, ns, ps, 
        fs, as, zs, ys, ks, Ms, Gs, Ts, Ps, Es, Zs, Ys
    Other: min, hr, D, W, M, Y, am, pm
    
Distance:
    Metric: m, dm, cm, mm, µm, nm, pm, km, 
        fm, am, zm, ym, Mm, Gm, Tm, Pm, Em, Zm, Ym
    Imperial: th, in, ft, yd, mi
    Other: nmi

Mass: 
    Metrix: mg, µg, ng, pg, kg, Mg, tonne
        dg, cg, fg, ag, zg, yg, dag, hg, Mg, Gg, Tg, Pg, Eg, Zg, Yg
    Imperial: oz, lb, ton
    Other: mol
    
Temparature: K, C, F, R

Luminosity: cd, lx

Force: N, kN, lbf, pdl

"""
None

# Classes

## Exceptions

### TokenNotAllowedException

In [4]:
class TokenNotAllowedException(Exception):
    def __init__(self, source, ln=None, offset=None):
        if ln == None and offset == None:
            ln = source.ln
            offset = source.offset
            source = source.source
            
        message = f"\n{source.getSourcePointerString(ln, offset)}"
        
        super().__init__(message)

### VariableNotDefined

In [5]:
class VariableNotDefined(Exception):
    def __init__(self, name):
        super().__init__(f"'{name}'")

### NotImplementedException

In [6]:
class NotImplementedException(Exception):
    pass

## Token Matching

### TokenOperandDefinition

In [7]:
class TokenOperandDefinition():
    """Defines an operand token that can be matched with using a string search"""
    
    def __init__(self, token, precedence):
        """Arguments:
            str   token:       string that identifies the operand
            float precedence:  precedence in order of operations (higher is executed first)
        """
        
        self.token = token
        self.precedence = float(precedence)
        
    def __repr__(self):
        return str(self)
        
    def __str__(self):
        return f'TokenOperandDefinition({self.token}, {self.precedence})'

### TokenGroupDefinition

In [8]:
class TokenGroupDefinition():
    """Defines an grouping token pair that can be matched with using a string search"""
    
    def __init__(self, open_token, close_token, seperators, ignore_missing, function):
        """Arguments:
            str      open_token:          string that identifies the opening token
            str      close_token:         string that identifies the closing token (can be the same as open_token)
            dict(str: depth) seperators:  allowed seperators and their seperating depth
            function function:            the function to convert the list of items into an in environment object
        """
        
        self.open_token = open_token
        self.close_token = close_token
        self.seperators = seperators
        self.ignore_missing = ignore_missing
        
        self.function = function
        
    def __repr__(self):
        return str(self)
        
    def __str__(self):
        return f'TokenGroupDefinition({self.open_token}, {self.close_token})'

### TokenNewItemDefinition

In [9]:
class TokenNewItemDefinition():
    """Defines an item seperation token that can be matched with using a string search"""
    
    def __init__(self, token):
        """Arguments:
            str token:  string that identifies the seperator
        """
        
        self.token = token
        
    def __repr__(self):
        return str(self)
        
    def __str__(self):
        return f'TokenNewItemDefinition({self.token}, {self.levels})'

### Source

In [10]:
class Source:
    
    def __init__(self, string=''):
        self.string = string
        self.lines = []
        
    def set(self, string):
        self.string = string
        self.lines = string.splitlines(True)
        
    def getSourcePointerString(self, ln, offset):
        ln_number_string = str(ln+1)
        line = f'{ln_number_string}: {self.lines[ln]}'
        pointer = ' ' * (len(ln_number_string) + 2 + offset) + '↑'
        sep = '' if '\n' in line[-2:] else '\n'
        
        return line + sep + pointer

### Token

In [11]:
class Token:
    
    def __init__(self, token_type, string, ln, offset, source):
        self.token_type = token_type
        self.string     = string
        self.ln         = ln
        self.offset     = offset
        self.source     = source
        
    def getSourcePointerString(self):
        return self.source.getSourcePointerString(self.ln, self.offset)

## Token Tree Nodes

### Abstract NodeToken

In [12]:
class NodeToken():
    """represents a value, operation or grouping node that returns an evaluabe that can be evaluated to return a value"""
    
    def is_complete(self):
        """Returns whether this token is complete, if false, it is not a valud token and interpreting failed"""
        
        raise NotImplementedException('NodeToken.is_complete()')
        
    def set_left(self, node):
        """sets the left child node if it exists"""
        
        raise NotImplementedException('NodeToken.set_left(node)')

    def set_right(self, node):
        """sets the right child node if it exists"""
        
        raise NotImplementedException('NodeToken.set_right(node)')
        
    def get_right(self):
        """segetsts the right child node if it exists"""
        
        raise NotImplementedException('NodeToken.get_right()')
        
    def get_evaluable(self):
        """return an Evaluable obejct from this token, the token should be complete before this method is called"""
        
        raise NotImplementedException('NodeToken.get_evaluable()')

### NodeBinary

In [13]:
class NodeBinary(NodeToken):
    """represents a binary operation with a left and right child"""
    
    def __init__(self, operation_definition, parent=None):
        self.parent = parent
        self.operation_definition = operation_definition
        self.precedence = operation_definition.precedence
        self.left = None
        self.right = None
    
    def is_complete(self):
        return self.left != None and self.right != None and self.left.is_complete() and self.right.is_complete()
        
    def set_left(self, node):
        """sets left child"""
        
        self.left = node
        node.parent = self

    def set_right(self, node):
        """sets right child"""
        
        self.right = node
        node.parent = self
        
    def get_right(self):
        """returns right child"""
        
        return self.right
    
    def get_evaluable(self):
        """returns Evaluable that computes a result from both child tokens"""
        
        return VariableFunction('<binary_operation>'+self.operation_definition.token, (self.left.get_evaluable(), self.right.get_evaluable()))
    
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        return f'NodeBinary({self.operation_definition}, left={self.left}, right={self.right})'

### NodeUnaryLeft

In [14]:
class NodeUnaryLeft(NodeToken):
    """represents a unary operator to the left of its operand with a single child"""
    
    def __init__(self, operation_definition, parent=None):
        self.parent = parent
        self.operation_definition = operation_definition
        self.precedence = operation_definition.precedence
        self.child = None
    
    def is_complete(self):
        return self.child != None and self.child.is_complete()
        
    def set_left(self, node):
        """sets single child"""
        
        self.child = node
        node.parent = self

    def set_right(self, node):
        """sets single child"""
        
        self.child = node
        node.parent = self
        
    def get_right(self):
        """gets single child"""
        
        return self.child
    
    def get_evaluable(self):
        """returns Evaluable that computes a result from the child token"""
        
        return VariableFunction('<left_unary_operation>'+self.operation_definition.token, (self.child.get_evaluable(), ))
    
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        return f'NodeUnaryLeft({self.operation_definition}, child={self.child})'

### NodeUnaryRight

In [15]:
class NodeUnaryRight(NodeToken):
    """represents a unary operator to the right of its operand with a single child"""
    
    def __init__(self, operation_definition, parent=None):
        self.parent = parent
        self.operation_definition = operation_definition
        self.precedence = operation_definition.precedence
        self.child = None
    
    def is_complete(self):
        return self.child != None and self.child.is_complete()
        
    def set_left(self, node):
        """sets single child"""
        
        self.child = node
        node.parent = self

    def set_right(self, node):
        """sets single child"""
        
        self.child = node
        node.parent = self
        
    def get_right(self):
        """gets single child"""
        
        return self.child
    
    def get_evaluable(self):
        """returns Evaluable that computes a result from the child token"""
        
        return VariableFunction('<right_unary_operation>'+self.operation_definition.token, (self.child.get_evaluable(), ))
    
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        return f'NodeUnaryRight({self.operation_definition}, child={self.child})'

### NodeGroup

In [16]:
class NodeGroup(NodeToken):
    """represents a grouping"""
    
    def __init__(self, group_definition, parent=None):
        self.parent = parent
        self.children = []
        
        self.group_definition = group_definition
        self.complete = False
        
        self.shape = {}
        self.sep_count = 0
    
    def is_complete(self, test_self=False):
        complete = test_self or self.complete
        complete &= all(c.is_complete() for c in self.children)
        return complete

    def close(self):
        self.sep_count = 0
        self.complete = True
        
        expected_size = 1
        for d,s in self.shape.items(): expected_size *= s
        
        if not self.group_definition.ignore_missing:
            assert len(self.children) == expected_size, f'Inconsistent dimensions shape={self.shape}, expected={expected_size}, actual={len(self.children)}'
    
    def set_right(self, node):
        self.children[-1] = node
        node.parent = self
        
    def get_right(self):
        return self.children[-1]

    def add(self, node):
        self.children.append(node)
        node.parent = self
        
        if len(self.shape) > 0 and self.sep_count >= len(self.shape):
            for i in range(len(self.shape), self.sep_count+1):
                self.shape[i-1] = self.shape.get(i-1, 1)
            self.shape[self.sep_count-1] += 1
            
        self.sep_count = 0
        
    def increase(self, depth=1):
        self.sep_count += depth
        
        if len(self.shape) == 0:
            self.shape[0] = 1
        
        # ensure minumum number of dimensions exists
        for i in range(len(self.shape), self.sep_count+1):
            self.shape[i-1] = self.shape.get(i-1, 1)
    
    def get_shape(self):
        shape = list(self.shape.items())
        shape.sort(key=lambda x:x[0], reverse=True)
        shape = tuple(s for d,s in shape)
        return shape
    
    def get_children(self):
        return self.children
    
    def get_evaluable(self):
        
        evaluables = [v.get_evaluable() for v in self.children]
        
        return VariableTensor(evaluables, self.get_shape(), self.group_definition.function)
    
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        return f'NodeGroup({self.group_definition}, children={self.children})'

### NodeValue

In [17]:
class NodeValue(NodeToken):
    """represents a value"""
    
    def __init__(self, value, value_type, parent=None):
        self.parent = parent
        self.value = value
        self.value_type = value_type
    
    def is_complete(self):
        return True
    
    def get_evaluable(self):
        """returns the approriate Evaluable for the tyoe of value this node holds"""

        value = None
        if self.value_type == TOKEN_TYPE_STRING:
            trimmed_string = self.value.strip(self.value[0])
            value = String(trimmed_string)
            value = VariableTensor([value], ())
            
        if self.value_type == TOKEN_TYPE_INTEGER:
            value = Integer(self.value)
            value = VariableTensor([value], ())
            
        if self.value_type == TOKEN_TYPE_NUMBER:
            value = Real(self.value)
            value = VariableTensor([value], ())
            
        if self.value_type == TOKEN_TYPE_LITERAL:
            value = Variable(self.value)
        
        return value
    
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        return f'NodeValue({self.value}, {self.value_type})'

## Evaluable Tree Nodes

### Abstract Evaluable

In [18]:
class Evaluable():
    """represents a node that can be evaluated"""
    
    def eval(self, environment, **kwargs):
        """performs the final evaluation returning an in environment value
        
        Arguments:
            Environment environment:  holds the current environment variables
                           **kwargs:  any arguments the Evaluable requires to evaluate
        """
        
        raise NotImplementedException('Evaluable.eval(environment)')

### VariableFunction

In [19]:
class VariableFunction(Evaluable):
    
    def __init__(self, name, parameters):
        
        self.dtype = Evaluable
        self.name = name
        self.parameters = parameters
        if isinstance(parameters, VariableTensor):
            self.parameters = parameters.data
    
    def eval(self, environment, evaluable=False, **kwargs):
        
        if evaluable:
            return self
        
        assert self.name in environment, f'Function {self.name} not found'
        assert type(environment[self.name]) == FunctionSet, f'{self.name} is not a function'
        
        return environment[self.name].eval(environment, self.parameters, **kwargs)
    
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        return f'VariableFunction_{self.name}({self.parameters})'

### Variable

In [20]:
class Variable(Evaluable):
    
    def __init__(self, name):
        
        self.dtype = Evaluable
        self.name = name
    
    def eval(self, environment, references=False, evaluable=False, prefix=None, **kwargs):
        
        prefix = '' if prefix == None else f'<{prefix}>'
        lookup_name = prefix + self.name
        
        if references:
            return Tensor(Reference(lookup_name), ())
        
        if evaluable:
            return self
        
        if lookup_name not in environment:
            raise VariableNotDefined(lookup_name)
        
        value = environment[lookup_name]
        
        if type(value) == Evaluable:
            value = value.eval(environment)
        
        return value
    
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        return f"Variable({self.name})"

### VariableTensor

In [21]:
class VariableTensor(Evaluable):
    
    def __init__(self, data, shape, function=None):
        
        self.dtype = Evaluable
        self.data = data
        self.shape = shape
        self.function = function
    
    def eval(self, environment, evaluable=False, **kwargs):
        
        if evaluable:
            return self
        
        values = [v.eval(environment, **kwargs) for v in self.data]
        
        if self.function == None and self.shape == ():
            return Tensor(values[:1], self.shape)
        else:
            return self.function(environment, values, self.shape, **kwargs)
    
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        return f'VariableTensor({self.data}, {self.shape})'

## Data Structures

### Tensor

In [22]:
def nd_index_from_shape(int_index, shape):
    devisor = 1
    devisors = [] if len(shape) == 0 else [devisor]
    for s in reversed(shape[1:]):
        devisor *= s
        devisors.append(devisor)
    index = []
    for devisor in reversed(devisors):
        i = int_index // devisor
        index.append(i)
        int_index -= i * devisor
    return index

In [23]:
def count_end_zeros(data, sub=None):
    if sub:
        data = [d-s+1 for [d,s] in zip(data, sub)]
    c = 0
    for i in reversed(data):
        if i == 0:
            c += 1
        else:
            break
    return c

In [24]:
class Tensor(Evaluable):
    """
    represents a value/tensor of any type, shape and rank
    """
    def __init__(self, data, shape=()):
        
        data = np.array(data, object)
        data = data.reshape((data.size,))
        
        assert data.size > 0, 'Tensor cannot be empty'
        
        if all(type(t) == Tensor for t in data):
            inner_shape = data[0].shape
            assert all(t.shape == inner_shape for t in data), 'Cannot combine tensors of different shapes'
            
            data = np.concatenate([t.data for t in data])
            shape = shape + inner_shape
        
        self._data = np.array(data, object).reshape(shape)
        self.data = self._data.reshape((self._data.size))
        
        self.first = self.data[0]
        self.shape = self._data.shape
        self.size = self._data.size
        self.rank = self._data.ndim
        self.dtype = type(self.data[0])
        
        assert all([type(i) == self.dtype for i in self.data]), 'All items in a Tensor must be of the same type'
        
    def __getitem__(self, index):
        
        data = self.data[index] if type(index) == int else self._data[index]
        shape = data.shape if type(data) == np.ndarray else ()
            
        return Tensor(data, shape)
        
    def __setitem__(self, index, value):
        
        if type(value) == Tensor:
            value = value._data
        
        if type(index) == int:
            self.data[index] = value
        else:
            self._data[index] = value
    
    def eval(self, environment, **kwargs):
        return self
    
    def __repr__(self):
        if self.rank == 0:
            return f'Tensor({self.data[()]})'
        else:
            return f'Tensor({self.data}, {self.shape})'
    
    def __str__(self):
        if self.rank == 0:
            return str(self.first)
        else:
            shape = list(self.shape)
            inner_size = shape.pop()
            strings = [str(i) for i in self.data]
            inner_lists = list(zip(*[iter(strings)]*inner_size))
            strings = ['['+', '.join(l)+']' for l in inner_lists]
            
            indexes = [nd_index_from_shape(i, shape) for i in range(len(strings))]
            
            left = [count_end_zeros(d) for d in indexes]
            left = [(len(shape) - i) * ' ' + i * '[' for i in left]

            right = [count_end_zeros(d, shape) for d in indexes]
            right = [i * ']' + ',' + i * '\n' for i in right]

            strings = [l+s+r for (l,s,r) in zip(left, strings, right)]
            strings[-1] = strings[-1].strip('\n,')

            string = '\n'.join(strings)
        
            return string

### Array

In [25]:
class Array(Evaluable):
    
    def __init__(self, data):
        self.data = np.array(data, object)
        self.data = self.data.reshape((self.data.size,))
        self.size = self.data.size
        self.dtype = Array
            
    def __getitem__(self, index):
        if type(index) == int:
            return self.data[index]
        elif type(index) in [tuple, list]:
            if len(index) == 0:
                raise Exception('No index given in array item lookup')
            elif len(index) == 1:
                return self.data[index[0]]
            else:
                raise Exception('Too many indices given in array item lookup')
        else:
            return Array(self.data[index])
        
    def __setitem__(self, index, value):
        self.data[index] = value
    
    def eval(self, environment, **kwargs):
        return self
    
    def __repr__(self):
        return f'Array({self.data[()]})'
    
    def __str__(self):
        items = [str(i) for i in self.data]
        items = ', '.join(items)
        return f'({items})'

### FunctionSet

In [26]:
class FunctionSet(Evaluable):
    
    def __init__(self, name, evaluation_paremeters=None):
        self.name = name
        self.signatures = {}
        self.evaluation_paremeters = evaluation_paremeters or {}
        
        self.dtype = FunctionSet
        
    def __contains__(self, partial_signature):
        l = len(partial_signature)
        for signature in self.signatures:
            if signature[:l] == partial_signature:
                return True
            
        return False
    
    def __getitem__(self, signature):
        return self.signatures[signature]
    
    def __setitem__(self, signature, value):
        par_names, function = value
        self.signatures[signature] = (par_names, function)
        
    def add(self, function_signature):
        self.signatures[function_signature.parameter_types] = function_signature
        
    def eval(self, environment, parameters, **kwargs):
        
        parameters = list(parameters)
        signature = [type(p) for p in parameters]
        
        for i, parameter in enumerate(parameters):
            eval_params = self.evaluation_paremeters.get(i, {})
            parameters[i] = parameter.eval(environment, **kwargs, **eval_params)
            signature[i] = parameters[i].dtype
        
        signature = tuple(signature)
        function_signature = None
        
        for signature_candidate in self.signatures:
            if len(signature) == len(signature_candidate) and all([a==b or a==None for a,b in zip(signature_candidate, signature)]):
                function_signature = self.signatures[signature_candidate]
        
#         assert signature in self.signatures, f'Signature {self.name}{tuple(p.__name__ for p in signature)} has no matching overload'
        assert function_signature != None, f'Signature {self.name}{tuple(p.__name__ for p in signature)} has no matching overload'
            
#         function_signature = self.signatures[signature]
        
        output_unit = None
        output_unit_instructions = function_signature.output_unit
        if output_unit_instructions != None:
            input_units = [p.first.unit if type(p) == Tensor else None for p in parameters]
            output_unit = Units()
            for instruction, input_unit in zip(output_unit_instructions, input_units):
                
                if type(instruction) == int:
                    if instruction == 1:
                        output_unit.add(input_unit)
                    if instruction == -1:
                        output_unit.sub(input_unit)
                    
                elif type(instruction) == bool:
                    if instruction == True:
                        if output_unit != input_unit:
                            raise Exception('Units not compatiple')
                            
                elif type(instruction) == Unit:
                    output_unit.add(instruction)
                            
                elif type(instruction) == Units:
                    output_unit.add(instruction)
        
            if output_unit.units == {}:
                output_unit = None
                    
    
    
        function = function_signature.function
        parameter_names = function_signature.parameter_names
        
        actual_shapes = [p.shape if type(p) == Tensor else () for p in parameters]
        actual_ranks = [len(s) for s in actual_shapes]
        
        ranks = [r if r!=None else a for r,a in zip(function_signature.parameter_ranks, actual_ranks)]
        extended_shapes = [s if r==0 else s[:-r] for s,r in zip(actual_shapes, ranks)]
        
        if len(ranks) == len(extended_shapes) == 0:
            extended_shape = []
        else:
            extended_shape = reduce(lambda x,y: x if len(x) > len(y) else y, extended_shapes)
        extended_ranks = [len(s) for s in extended_shapes]
        
        assert all([s in [extended_shape, ()] for s in extended_shapes]), f'Ranks, shape or extended shapes do not match'
        
        if extended_shape == ():
            return self.runFunction(function, environment, parameter_names, parameters, output_unit)
        
        else:
            data = np.empty(extended_shape, dtype=object)
            
            for i in product(*[range(d) for d in extended_shape]):
                extracted_parameters = [p if r == 0 else p[i] for p, r in zip(parameters, extended_ranks)]
                
                data[i] = self.runFunction(function, environment, parameter_names, extracted_parameters, output_unit)._data.tolist()
                    
            data = np.array(data.tolist())
            
            return Tensor(data, data.shape)
        
    def runFunction(self, function, environment, parameter_names, parameters, output_unit):
        
        if parameter_names == False or parameter_names == None:
            results = function(*parameters)
        
        elif parameter_names == True:
            results = function(environment, *parameters)

        else:
            new_scoped_environment = environment.enterScope({k:v for k,v in zip(parameter_names, parameters)})
            results = function(new_scoped_environment)
        
        if output_unit != None:
            for value in results.data:
                value.unit = Units(output_unit)
            
        return results
        
    def attach_units(self, value, unit):
        pass
        
    def __repr__(self):
        return f'FunctionSet({self.name}, {self.signatures})'
    
    def __str__(self):
        if len(self.signatures) == 0:
            parameter_types = ''
        if len(self.signatures) == 1:
            parameter_types = next(iter(self.signatures))
            parameter_types = ['Any' if t == None else t.__name__ for t in parameter_types]
            parameter_types = ', '.join(parameter_types)
        else:
            parameter_types = '...'
            
        name = '<AnonymousFunction>' if self.name == '' else self.name
            
        return f'{name}({parameter_types})'

### ConversionFunctionSet (DEPRICATED)

In [27]:
# class ConversionFunctionSet(Evaluable):
    
#     def __init__(self, name, forwards, backwards):
#         self.name = name
#         self.forwards = forwards
#         self.backwards = backwards
#         self.dtype = ConversionFunctionSet
    
#     def eval(self, environment, tensor, forwards=True, **kwargs):
#         shape = tensor.shape
#         data = tensor.data
        
#         if forwards:
#             converted_data = [self.forwards(d) for d in data]
#         else:
#             converted_data = [self.backwards(d) for d in data]
            
#         return Tensor(converted_data, shape)
    
#     def __repr__(self):
#         return f'ConversionFunctionSet({self.name})'
    
#     def __str__(self):
#         return f'<{self.name}>'

## Data Types

### Abstract Data

In [28]:
class Data(Evaluable):
    pass

### String

In [29]:
class String(Data):
    
    def __init__(self, value):
        self.value = value
    
    def eval(self, environment, **kwargs):
        return self
    
    def __repr__(self):
        return f'String("{self.value}")'
    
    def __str__(self):
        return f'"{self.value}"'

### Integer

In [30]:
class Integer(Data):
    
    def __init__(self, value):
        self.value = value
    
    def eval(self, environment, **kwargs):
        return self

### Real

In [31]:
class Real(Data):
    
    def __init__(self, value, unit=None):
        self.value = Decimal(value)
        self.unit = unit
    
    def eval(self, environment, **kwargs):
        return self
    
    def __repr__(self):
        return f'Real({self.value}, {self.unit})'
    
    def __str__(self):
        if self.unit == None:
            return f'{self.value}'
        else:
            value = self.unit.convert_backwards(Tensor(self))
            return f'{value}{self.unit}'

### Complex

### Boolean

In [32]:
class Boolean(Data):
    
    def __init__(self, value):
        self.value = value
    
    def eval(self, environment, **kwargs):
        return self

### FunctionSignature

In [33]:
class FunctionSignature():
    
    def __init__(self, name, function, parameter_names, parameter_types, parameter_ranks=None, parameter_shapes=None, output_unit=None):
        
        self.name = name
        self.function = function
        self.parameter_count = len(parameter_types)
        self.parameter_names = parameter_names
        self.parameter_types = parameter_types
        self.parameter_ranks = parameter_ranks or self.parameter_count*(None,)
        self.parameter_shapes = parameter_shapes or self.parameter_count*(None,)
        self.output_unit     = output_unit
        
    def __repr__(self):
        return f'FunctionSignature{(self.name, self.function, self.parameter_names, self.parameter_types, self.parameter_ranks, self.parameter_shapes, self.output_unit)}'
    
    def __str__(self):
        parameter_types = [str(t) for t in self.parameter_types]
        parameter_types = ', '.join(parameter_types)
        return f'{self.name}({parameter_types})'

### Reference

In [34]:
class Reference(Data):
    
    def __init__(self, value):
        self.value = value
    
    def eval(self, environment, **kwargs):
        return self

## Units

### Unit

In [35]:
class Unit:
    
    def __init__(self, group_name, unit_name, forwards, backwards):
        self.group_name = group_name
        self.unit_name = unit_name
        self.forwards = forwards
        self.backwards = backwards
        
        self.dtype = Unit
        
    def convert_forwards(self, tensor):
        return self.convert(tensor, self.forwards)
        
    def convert_backwards(self, tensor):
        return self.convert(tensor, self.backwards)
        
    def convert(self, tensor, converter):
        if callable(converter):
            return converter(tensor)
        else:
            return Tensor(Real(tensor.first.value * converter))
    
    def get_base_units(self):
        return {self: 1}
        
    def __hash__(self):
        return hash(self.group_name)
    
    def __eq__(self, other):
        if type(other) != Unit:
            return False
        return self.group_name == other.group_name
        
    def __repr__(self):
        return f'Unit({self.group_name}, {self.unit_name})'
        
    def __str__(self):
        return self.unit_name

### Units (Unit collection)

In [36]:
class Units:
    
    def __init__(self, units=None, name=None):
        
        if type(units) == Units:
            print('Units -> Units', units, name)
            if units.name == None:
                self.units = {**units.units}
                self.name = units.name
            else:
                print('child')
                self.units = {units: 1}
                self.name = name
                
        elif type(units) == Unit:
            self.units = {units: 1}
            self.name = name
            
        else:
            self.units = units or {}
            self.name = name
        
        self.dtype = Unit
        
    def convert_forwards(self, tensor):
        tensor = tensor
        
        for unit, degree in self.units.items():
            if degree > 0:
                for i in range(degree):
                    tensor = unit.convert_forwards(tensor)
            else:
                for i in range(-degree):
                    tensor = unit.convert_backwards(tensor)
        
        return tensor
        
    def convert_backwards(self, tensor):
        tensor = tensor
        
        for unit, degree in self.units.items():
            if degree > 0:
                for i in range(degree):
                    tensor = unit.convert_backwards(tensor)
            else:
                for i in range(degree):
                    tensor = unit.convert_forwards(tensor)
        
        return tensor
    
    def get_base_units(self):
        base_units = {}
        for [unit, degree] in self.units.items():
            units = unit.get_base_units()
            for [inner_unit, inner_degree] in units.items():
                base_units[inner_unit] = base_units.get(inner_unit, 0) + inner_degree * degree
                
        return base_units
        
    def __hash__(self):
        return hash(self.group_name)
    
    def __eq__(self, other):
        if other == None:
            return self.units == {}
        return self.get_base_units() == other.get_base_units()

    def add(self, other, super_degree=1):
        
        new_units = {}
        if other == None:
            return
        elif type(other) == Unit:
            new_units[other] = self.units.get(other, 0) + super_degree
        elif type(other) == Units:
            for unit, degree in other.units.items():
                new_units[unit] = self.units.get(unit, 0) + super_degree * degree
            
        self.units = {**self.units, **new_units}
        
        to_be_removed = [unit for unit, degree in self.units.items() if degree == 0]
        for unit in to_be_removed:
            del self.units[unit]
    
    def sub(self, other, super_degree=1):
        self.add(other, -super_degree)
        
    def __repr__(self):
        return f'Units({self.name}, {self.units})'
        
    def __str__(self):
        if self.name == None:
            string_positive = ''
            string_negative = ''
            
            for unit, index in self.units.items():
                super_script = ''
                if index < -1 or index > 1:
                    super_script = '^'+str(abs(index))
                    
                if index > 0:
                    string_positive += f'*{unit}{super_script}'
                if index < 0:
                    string_negative += f'/{unit}{super_script}'
                
            return string_positive.strip('*') + string_negative
        
        else:
            return self.name

## BuiltIns

In [37]:
class BuiltIns():
    
    def __init__(self):
        self.functions = []
        self.function_parameter_evaluation_parameters = {}
        self.vars = []
        self.binary_operators = []
        self.binary_operators_continuing = []
        self.left_unary_operators = []
        self.right_unary_operators = []
        self.groups = []
        self.new_items = []
        self.conversion_function_sets = []
        self.units = []
        
    def register_function(self, 
                          name, 
                          function, 
                          parameter_names, 
                          parameter_types, 
                          parameter_ranks=None, 
                          parameter_shapes=None,
                          output_unit=None
                         ):
        self.functions.append((
            name, 
            function, 
            parameter_names, 
            parameter_types, 
            parameter_ranks, 
            parameter_shapes,
            output_unit))
        
    def register_operation_function(self, 
                                    operation, 
                                    name, 
                                    function, 
                                    parameter_names, 
                                    parameter_types, 
                                    parameter_ranks=None, 
                                    parameter_shapes=None,
                                    output_unit=None
                                   ):
        names = name if type(name) == list else [name]
        for name in names:
            self.functions.append((
                f'<{operation}_operation>{name}', 
                function, 
                parameter_names, 
                parameter_types, 
                parameter_ranks, 
                parameter_shapes,
                output_unit))
        
    def set_evaluation_parameter(self, function_name, operation, parameter_index, key, value):
        function_name = f'<{operation}_operation>{function_name}'
        parameter_list = self.function_parameter_evaluation_parameters.get(function_name, {})
        parameters = parameter_list.get(parameter_index, {})
        parameters[key] = value
        parameter_list[parameter_index] = parameters
        self.function_parameter_evaluation_parameters[function_name] = parameter_list
        
    def register_binary_operator(self, name, precedence, continuing=False):
        if continuing:
            self.binary_operators_continuing.append(TokenOperandDefinition(name, precedence))
        
        self.binary_operators.append(TokenOperandDefinition(name, precedence))
        
    def register_left_unary_operator(self, name, precedence):
        self.left_unary_operators.append(TokenOperandDefinition(name, precedence))
        
    def register_right_unary_operator(self, name, precedence):
        self.right_unary_operators.append(TokenOperandDefinition(name, precedence))
        
    def register_var(self, name, value):
        self.vars.append((name, value))
        
    def register_grouping(self, open_token, close_token, seperators, ignore_missing, function):
        self.groups.append(TokenGroupDefinition(open_token, close_token, seperators, ignore_missing, function))
        
    def register_new_item_seperator(self, token):
        self.new_items.append(TokenNewItemDefinition(token))
        
    def register_conversion_function_set(self, name, forwards, backwards):
        self.conversion_function_sets.append((name, forwards, backwards))
        
    def register_unit(self, unit_name, unit):
        self.units.append((unit_name, unit))
        
    def register_unit_group(self, group_name, unit_group):
        for [unit_name, converters] in unit_group.items():
            self.register_unit(
                unit_name,
                Unit(
                    group_name, 
                    unit_name, 
                    converters['forwards'], 
                    converters['backwards']
                )
            )
        
        
built_ins = BuiltIns()

# Built In Functions, Operators and Variables

### Groups

#### Group ( )

In [38]:
def group_round(environment, items, shape, force_array=False, **kwargs):
    
    if shape == () and len(items) > 0 and force_array == False:
        return items[0]
    else:
        return Array(items)

built_ins.register_grouping('(', ')', {',': 1, '\n': 1}, True, group_round)

#### Group [ ]

In [39]:
def group_squre(environment, items, shape, **kwargs):
    
    assert len(items) > 0, 'Tensors can not be empty'
    
    base_shape = items[0].shape
    assert all([i.shape == base_shape for i in items]), 'Tensor elements have inconsistent shapes'
    
    extended_shape = shape + base_shape
    data = np.array([i.data for i in items]).reshape(extended_shape)
    
    return Tensor(data, extended_shape)

built_ins.register_grouping('[', ']', {',': 1, ';': 2, '\n': 0}, False, group_squre)

#### Group { }

In [40]:
def group_curly(environment, items, shape, **kwargs):
    return items[-1]

built_ins.register_grouping('{', '}', {';': 1, '\n': 1}, True, group_curly)

#### Group | |

In [41]:
def group_straight(environment, items, shape, **kwargs):
    
    assert shape == (), 'Cannot take absolute value of an array'
    assert items[0].dtype == Real, 'Cannot take absolute value of non Real value'
    
#     return Tensor(Real(abs(items[0].first.value)))

    return VariableFunction('abs', items).eval(environment)

built_ins.register_grouping('|', '|', {}, True, group_straight)
built_ins.register_grouping('||', '||', {}, True, group_straight)

### New Item Seperators

In [42]:
built_ins.register_new_item_seperator(',')
built_ins.register_new_item_seperator(';')
built_ins.register_new_item_seperator('\n')

### Units

#### Distance

In [43]:
distance = {
    'km': {
        'forwards':  Decimal('1000'),
        'backwards': Decimal('.001'),
    },
    'm': {
        'forwards':  Decimal('1'),
        'backwards': Decimal('1'),
    },
    'dm': {
        'forwards':  Decimal('.1'),
        'backwards': Decimal('10'),
    },
    'cm': {
        'forwards':  Decimal('.01'),
        'backwards': Decimal('100'),
    },
    'mm': {
        'forwards':  Decimal('.001'),
        'backwards': Decimal('1000'),
    },
    'um': {
        'forwards':  Decimal('.000001'),
        'backwards': Decimal('1000000'),
    },
    'nm': {
        'forwards':  Decimal('.000000001'),
        'backwards': Decimal('1000000000'),
    },
    'pm': {
        'forwards':  Decimal('.000000000001'),
        'backwards': Decimal('1000000000000'),
    },
    'in': {
        'forwards':  Decimal('0.0254'),
        'backwards': Decimal('39.37007874015748031496062992'),
    },
    'tf': {
        'forwards':  Decimal('0.3048'),
        'backwards': Decimal('3.280839895013123359580052493'),
    },
    'yd': {
        'forwards':  Decimal('0.9144'),
        'backwards': Decimal('1.093613298337707786526684164'),
    },
    'mi': {
        'forwards':  Decimal('1609.344'),
        'backwards': Decimal('0.0006213711922373339696174341844'),
    },
}

dm = Unit(
    'distance', 
    'dm', 
    distance['dm']['forwards'], 
    distance['dm']['backwards']
)

cm = Unit(
    'distance', 
    'cm', 
    distance['cm']['forwards'], 
    distance['cm']['backwards']
)

built_ins.register_unit_group('distance', distance)

#### Volume

In [44]:
L  = Units({dm: 3}, 'L')
mL = Units({cm: 3}, 'mL')
cc = Units({cm: 3}, 'cc')

built_ins.register_unit('L', L)
built_ins.register_unit('mL', mL)
built_ins.register_unit('cc', cc)

#### Time

In [45]:
def time_formart_forwards(string):
    pass

def time_formart_backwards(seconds):
    pass

time = {
    'd': {
        'forwards':  Decimal('86400'),
        'backwards': Decimal('.00001157407407407407407407407407'),
    },
    'hr': {
        'forwards':  Decimal('3600'),
        'backwards': Decimal('.0002777777777777777777777777778'),
    },
    'min': {
        'forwards':  Decimal('60'),
        'backwards': Decimal('.01666666666666666666666666667'),
    },
    's': {
        'forwards':  Decimal('1'),
        'backwards': Decimal('1'),
    },
    'ms': {
        'forwards':  Decimal('.001'),
        'backwards': Decimal('1000'),
    },
    'us': {
        'forwards':  Decimal('.000001'),
        'backwards': Decimal('1000000'),
    },
    'ns': {
        'forwards':  Decimal('.000000001'),
        'backwards': Decimal('1000000000'),
    },
    'time': {
        'forwards':  Decimal('1'),
        'backwards': Decimal('1'),
        'formart_forwards':  time_formart_forwards,
        'formart_backwards': time_formart_backwards,
    }
}

_min = Unit(
    'time',
    'min',
    time['min']['forwards'],
    time['min']['backwards'],
)

built_ins.register_unit_group('time', time)

#### Angles

In [46]:
angle = {
    'deg': {
        'forwards':  Decimal('0.01745329251994329576923690768'),
        'backwards': Decimal('57.29577951308232087679815483'),
    },
    'grad': {
        'forwards':  Decimal('0.01570796326794896619231321692'),
        'backwards': Decimal('63.66197723675813430755350533'),
    },
    'rad': {
        'forwards':  Decimal('1'),
        'backwards': Decimal('1'),
    },
    'rev': {
        'forwards':  Decimal('6.283185307179586476925286767'),
        'backwards': Decimal('0.1591549430918953357688837634'),
    },
}

rad = Unit(
    'angle', 
    'rad', 
    angle['rad']['forwards'], 
    angle['rad']['backwards'],
)

rev = Unit(
    'angle', 
    'rev', 
    angle['rev']['forwards'], 
    angle['rev']['backwards'],
)

built_ins.register_unit_group('angle', angle)

#### Angular Speeds

In [47]:
rpm = Units({rev: 1, _min: -1}, 'rpm')

built_ins.register_unit('rpm', rpm)

#### 

### Variabels

In [48]:
built_ins.register_var('PI', Tensor([Real('3.141592653589793238462643383279502884197')]))
built_ins.register_var('e', Tensor([Real('2.718281828459045235360287471352662497757')]))
built_ins.register_var('phi', Tensor([Real('1.61803398874989484820458683436563811772')]))

### Operations Tokens

In [49]:
# Unary Operations: -, +
built_ins.register_left_unary_operator('+', 7)
built_ins.register_left_unary_operator('-', 7)
built_ins.register_left_unary_operator('=', 0)
built_ins.register_right_unary_operator('!', 7)

# Binary Operations:

# arithmetic
built_ins.register_binary_operator('+',   3, True)
built_ins.register_binary_operator('-',   3, True)
built_ins.register_binary_operator('*',   4, True)
built_ins.register_binary_operator('4',   4)
built_ins.register_binary_operator('/',   4, True)
built_ins.register_binary_operator('//',  4, True)
# built_ins.register_binary_operator('%',   4, True)
built_ins.register_binary_operator('mod', 4, True)
built_ins.register_binary_operator('^',   5, True)

# matrix
built_ins.register_binary_operator('#',   4, True)
built_ins.register_binary_operator('matmul', 4, True)

# vector
built_ins.register_binary_operator('.*' , 4, True)
built_ins.register_binary_operator('dot', 4, True)

# root
built_ins.register_left_unary_operator('$', 6)
built_ins.register_binary_operator('$', 6)

# functions as operators
built_ins.register_left_unary_operator('ln', 2)
built_ins.register_left_unary_operator('log2', 2)
built_ins.register_left_unary_operator('log10', 2)

# trigonometry
built_ins.register_left_unary_operator('sin', 2)
built_ins.register_left_unary_operator('cos', 2)
built_ins.register_left_unary_operator('tan', 2)
built_ins.register_left_unary_operator('asin', 2)
built_ins.register_left_unary_operator('acos', 2)
built_ins.register_left_unary_operator('atan', 2)

# unit
built_ins.register_binary_operator('implicit_unit', 9)
built_ins.set_evaluation_parameter('implicit_unit', 'binary', 1, 'prefix', 'unit')

# function call
built_ins.register_binary_operator('implicit_call', 9)
built_ins.set_evaluation_parameter('implicit_call', 'binary', 1, 'force_array', True)
built_ins.register_binary_operator('?', 7)

# function definition
built_ins.register_binary_operator('=>', 1)
built_ins.set_evaluation_parameter('=>', 'binary', 0, 'references', True)
built_ins.set_evaluation_parameter('=>', 'binary', 1, 'evaluable', True)

built_ins.register_binary_operator('=', 0)
built_ins.set_evaluation_parameter('=', 'binary', 0, 'references', True)
built_ins.set_evaluation_parameter('=', 'left_unary', 0, 'references', True)

# conversion
built_ins.register_binary_operator('@', -1, True)
built_ins.set_evaluation_parameter('@', 'binary', 1, 'prefix', 'unit')

### Operation Functions

#### Assignment: =

In [50]:
def assignment(environment, key, value):
    
    if type(key) == Array:
        for k, v in zip(key, value):
            assignment(environment, k, v)
    else:
        environment[key.first.value] = value
        
    return value

def post_assignment(environment, key):
    value = environment['']
    return assignment(environment, key, value)


built_ins.register_operation_function(
    operation        = 'binary', 
    name             = '=', 
    function         = assignment, 
    parameter_names  = True, 
    parameter_types  = (Array, None), 
    parameter_ranks  = None, 
    parameter_shapes = None,
    output_unit      = None
)
built_ins.register_operation_function(
    operation        = 'binary', 
    name             = '=', 
    function         = assignment, 
    parameter_names  = True, 
    parameter_types  = (Reference, None), 
    parameter_ranks  = None, 
    parameter_shapes = None,
    output_unit      = None
)
built_ins.register_operation_function(
    operation        = 'left_unary', 
    name             = '=', 
    function         = post_assignment, 
    parameter_names  = True, 
    parameter_types  = (Reference,), 
    parameter_ranks  = None, 
    parameter_shapes = None,
    output_unit      = None
)


#### Function Definition: =>

In [51]:
def define_function(environment, parameters, evaluable):
    
    if type(parameters) == Tensor and parameters.dtype == Reference:
        parameter_names = [parameters.first.value]
    elif type(parameters) == Array:
        parameter_names = [p.first.value for p in parameters.data]
    elif type(parameters) == tuple:
        parameter_names = list(parameters)
    
    parameter_types = tuple([None] * len(parameter_names))
    
    def function(environment):
        return evaluable.eval(environment)
    
    function_signature = FunctionSignature('', function, parameter_names, parameter_types, parameter_ranks=None, parameter_shapes=None)

    function_set = FunctionSet('')
    function_set.add(function_signature)
    
    return function_set

built_ins.register_operation_function(
    operation        = 'binary', 
    name             = '=>', 
    function         = define_function, 
    parameter_names  = True, 
    parameter_types  = (Reference, None), 
    parameter_ranks  = None, 
    parameter_shapes = None,
    output_unit      = None
)
built_ins.register_operation_function(
    operation        = 'binary', 
    name             = '=>', 
    function         = define_function, 
    parameter_names  = True, 
    parameter_types  = (Array, None), 
    parameter_ranks  = None, 
    parameter_shapes = None,
    output_unit      = None
)


#### Unary Operations: -, +

In [52]:
built_ins.register_operation_function(
    operation        = 'left_unary', 
    name             = '+', 
    function         = lambda t:Tensor(Real(+t.first.value)),
    parameter_names  = None, 
    parameter_types  = (Real,), 
    parameter_ranks  = (0,), 
    parameter_shapes = None,
    output_unit      = (1,)
)
built_ins.register_operation_function(
    operation        = 'left_unary', 
    name             = '-', 
    function         = lambda t:Tensor(Real(-t.first.value)),
    parameter_names  = None, 
    parameter_types  = (Real,), 
    parameter_ranks  = (0,), 
    parameter_shapes = None,
    output_unit      = (1,)
)


#### Binary Operantors: +, -, *, /, //, %, ^

In [53]:
# A + B
built_ins.register_operation_function(
    operation        = 'binary', 
    name             = '+', 
    function         = lambda a, b: Tensor(Real(a.first.value + b.first.value)), 
    parameter_names  = None, 
    parameter_types  = (Real, Real), 
    parameter_ranks  = (0, 0), 
    parameter_shapes = None,
    output_unit      = (1, True)
)

# A - B
built_ins.register_operation_function(
    operation        = 'binary', 
    name             = '-', 
    function         = lambda a, b: Tensor(Real(a.first.value - b.first.value)), 
    parameter_names  = None, 
    parameter_types  = (Real, Real), 
    parameter_ranks  = (0, 0), 
    parameter_shapes = None,
    output_unit      = (1, True)
)

# A * B
built_ins.register_operation_function(
    operation        = 'binary', 
    name             = '*', 
    function         = lambda a, b: Tensor(Real(a.first.value * b.first.value)), 
    parameter_names  = None, 
    parameter_types  = (Real, Real), 
    parameter_ranks  = (0, 0), 
    parameter_shapes = None,
    output_unit      = (1, 1)
)

# A / B
built_ins.register_operation_function(
    operation        = 'binary', 
    name             = '/', 
    function         = lambda a, b: Tensor(Real(a.first.value / b.first.value)), 
    parameter_names  = None, 
    parameter_types  = (Real, Real), 
    parameter_ranks  = (0, 0), 
    parameter_shapes = None,
    output_unit      = (1, -1)
)

# A // B
built_ins.register_operation_function(
    operation        = 'binary', 
    name             = '//', 
    function         = lambda a, b: Tensor(Real(a.first.value // b.first.value)), 
    parameter_names  = None, 
    parameter_types  = (Real, Real), 
    parameter_ranks  = (0, 0), 
    parameter_shapes = None,
    output_unit      = (1, -1)
)

# A % B
built_ins.register_operation_function(
    operation        = 'binary', 
    name             = 'mod', 
    function         = lambda a, b: Tensor(Real(a.first.value % b.first.value)), 
    parameter_names  = None, 
    parameter_types  = (Real, Real), 
    parameter_ranks  = (0, 0), 
    parameter_shapes = None,
    output_unit      = (1, None)
)

# A ^ B
built_ins.register_operation_function(
    operation        = 'binary', 
    name             = '^', 
    function         = lambda a, b: Tensor(Real(a.first.value ** b.first.value)), 
    parameter_names  = None, 
    parameter_types  = (Real, Real), 
    parameter_ranks  = (0, 0), 
    parameter_shapes = None,
    output_unit      = (1, None)
)


#### Binary Unit Operantors: *, /, ^

In [54]:
def units_add(unit1, unit2):
    new_unit = Units()
    new_unit.add(unit1)
    new_unit.add(unit2)
    return new_unit
    
# U1 * U2
built_ins.register_operation_function(
    operation        = 'binary', 
    name             = '*', 
    function         = units_add, 
    parameter_names  = None, 
    parameter_types  = (Unit, Unit), 
    parameter_ranks  = (0, 0), 
    parameter_shapes = None,
    output_unit      = (1, 1)
)

def units_sub(unit1, unit2):
    new_unit = Units()
    new_unit.add(unit1)
    new_unit.sub(unit2)
    return new_unit

# U1 / U2
built_ins.register_operation_function(
    operation        = 'binary', 
    name             = '/', 
    function         = units_sub, 
    parameter_names  = None, 
    parameter_types  = (Unit, Unit), 
    parameter_ranks  = (0, 0), 
    parameter_shapes = None,
    output_unit      = (1, -1)
)

def units_power(unit, tensor):
    new_unit = Units()
    new_unit.add(unit, int(tensor.first.value))
    return new_unit
    
# U ^ B
built_ins.register_operation_function(
    operation        = 'binary', 
    name             = '^', 
    function         = units_power, 
    parameter_names  = None, 
    parameter_types  = (Unit, Real), 
    parameter_ranks  = (0, 0), 
    parameter_shapes = None,
    output_unit      = None
)


#### Matrix Multiplication: # #

In [55]:
def matmul(A, B):
    
    a_shape = A.shape
    b_shape = B.shape
    
    assert A.rank >= 2 and B.rank >= 2, 'left and right sides must be matrices'
    assert a_shape[-1] == b_shape[-2], 'left n_cols must equal right n_rows'
    
    n_rows = a_shape[0]
    n_cols = b_shape[1]
    n_vec = a_shape[1]
    shape = (n_rows , n_cols)
    
    data = np.empty(shape, dtype=object)
    
    for row in range(n_rows):
        for col in range(n_cols):
            v = 0
            for i in range(n_vec):
                v += A[(row, i)].first.value * B[(i, col)].first.value
            data[row,col] = Real(v)
    
    return Tensor(data, shape)

built_ins.register_operation_function(
    operation        = 'binary', 
    name             = ['#', 'matmul'], 
    function         = matmul, 
    parameter_names  = None, 
    parameter_types  = (Real, Real), 
    parameter_ranks  = (2, 2), 
    parameter_shapes = None
)


#### Vector Dot Product: .*

In [56]:
def dot(A, B):
    
    a_shape = A.shape
    b_shape = B.shape
    
    assert A.rank == 1 and B.rank == 1, 'left and right sides must be vectors'
    assert a_shape == b_shape, 'vector length must match'
    
    n_vec = a_shape[-1]
    
    v = 0
    for i in range(n_vec):
        v += A[(i,)].first.value * B[(i,)].first.value
    
    return Tensor(Real(v))

built_ins.register_operation_function(
    operation        = 'binary', 
    name             = ['.*', 'dot'], 
    function         = dot, 
    parameter_names  = None, 
    parameter_types  = (Real, Real), 
    parameter_ranks  = (1, 1), 
    parameter_shapes = None
)


#### Root and Square root: $

In [57]:
def root(A, B):
    return Tensor(Real(B.first.value ** (Decimal(1) / A.first.value)))

def sqrt(A):
    return Tensor(Real(A.first.value ** Decimal(0.5)))

built_ins.register_operation_function(
    operation        = 'left_unary', 
    name             = '$', 
    function         = sqrt, 
    parameter_names  = None, 
    parameter_types  = (Real,), 
    parameter_ranks  = (0,), 
    parameter_shapes = None,
    output_unit      = (1,)
)
built_ins.register_operation_function(
    operation        = 'binary', 
    name             = '$', 
    function         = root, 
    parameter_names  = None, 
    parameter_types  = (Real, Real), 
    parameter_ranks  = (0, 0), 
    parameter_shapes = None,
    output_unit      = (None, 1)
)


#### Logarithm: ln, log10, log2, log(b, v)

In [58]:
def log2(A):
    return Tensor(Real( math.log2( A.first.value ) ))
    
def log10(A):
    return Tensor(Real( math.log10( A.first.value ) ))
    
def ln(A):
    return Tensor(Real( math.log( A.first.value ) ))
    
built_ins.register_operation_function(
    operation        = 'left_unary', 
    name             = 'log2', 
    function         = log2, 
    parameter_names  = None, 
    parameter_types  = (Real,), 
    parameter_ranks  = (0,), 
    parameter_shapes = None,
    output_unit      = (1, None)
)
    
built_ins.register_operation_function(
    operation        = 'left_unary', 
    name             = 'log10', 
    function         = log10, 
    parameter_names  = None, 
    parameter_types  = (Real,), 
    parameter_ranks  = (0,), 
    parameter_shapes = None,
    output_unit      = (1, None)
)
    
built_ins.register_operation_function(
    operation        = 'left_unary', 
    name             = 'ln', 
    function         = ln, 
    parameter_names  = None, 
    parameter_types  = (Real,), 
    parameter_ranks  = (0,), 
    parameter_shapes = None,
    output_unit      = (1, None)
)

#### Trigonometry

In [59]:
def sin(A):
    return Tensor(Real( math.sin( A.first.value ) ))

def cos(A):
    return Tensor(Real( math.cos( A.first.value ) ))

def tan(A):
    return Tensor(Real( math.tan( A.first.value ) ))
  
built_ins.register_operation_function(
    operation        = 'left_unary', 
    name             = 'sin', 
    function         = sin, 
    parameter_names  = None, 
    parameter_types  = (Real,), 
    parameter_ranks  = (0,), 
    parameter_shapes = None,
    output_unit      = (None,)
)
  
built_ins.register_operation_function(
    operation        = 'left_unary', 
    name             = 'cos', 
    function         = cos, 
    parameter_names  = None, 
    parameter_types  = (Real,), 
    parameter_ranks  = (0,), 
    parameter_shapes = None,
    output_unit      = (None,)
)
  
built_ins.register_operation_function(
    operation        = 'left_unary', 
    name             = 'tan', 
    function         = tan, 
    parameter_names  = None, 
    parameter_types  = (Real,), 
    parameter_ranks  = (0,), 
    parameter_shapes = None,
    output_unit      = (None,)
)

In [60]:
def asin(A):
    return Tensor(Real( math.asin( A.first.value ) ))

def acos(A):
    return Tensor(Real( math.acos( A.first.value ) ))

def atan(A):
    return Tensor(Real( math.atan( A.first.value ) ))
  
built_ins.register_operation_function(
    operation        = 'left_unary', 
    name             = 'asin', 
    function         = asin, 
    parameter_names  = None, 
    parameter_types  = (Real,), 
    parameter_ranks  = (0,), 
    parameter_shapes = None,
    output_unit      = (rad,)
)
  
built_ins.register_operation_function(
    operation        = 'left_unary', 
    name             = 'acos', 
    function         = acos, 
    parameter_names  = None, 
    parameter_types  = (Real,), 
    parameter_ranks  = (0,), 
    parameter_shapes = None,
    output_unit      = (rad,)
)
  
built_ins.register_operation_function(
    operation        = 'left_unary', 
    name             = 'atan', 
    function         = atan, 
    parameter_names  = None, 
    parameter_types  = (Real,), 
    parameter_ranks  = (0,), 
    parameter_shapes = None,
    output_unit      = (rad,)
)

#### Factorial: !

In [61]:
def factorial(A):
    count = int(A.first.value)
    value = Decimal(1)
    for i in range(1, count+1):
        value *= Decimal(i)
    
    return Tensor(Real(value))

built_ins.register_operation_function(
    operation        = 'right_unary', 
    name             = '!', 
    function         = factorial, 
    parameter_names  = None, 
    parameter_types  = (Real,), 
    parameter_ranks  = (0,), 
    parameter_shapes = None
)

#### Function Call

In [62]:
def function_call(environment, function_set, parameters):
    return function_set.eval(environment, parameters)

def piped_function_call(environment, parameters, function_set):
    return function_call(environment, function_set, parameters)

built_ins.register_operation_function(
    operation        = 'binary', 
    name             = 'implicit_call', 
    function         = function_call, 
    parameter_names  = True, 
    parameter_types  = (FunctionSet, Array), 
    parameter_ranks  = None, 
    parameter_shapes = None
)
built_ins.register_operation_function(
    operation        = 'binary', 
    name             = '?', 
    function         = piped_function_call, 
    parameter_names  = True, 
    parameter_types  = (None, FunctionSet), 
    parameter_ranks  = None, 
    parameter_shapes = None
)

#### Tensor Lookup

In [63]:
def tensor_lookup(environment, tensor, parameters):
    index = [int(p.first.value) for p in parameters]
    return tensor[tuple(index)]

built_ins.register_operation_function(
    operation        = 'binary', 
    name             = 'implicit_call', 
    function         = tensor_lookup, 
    parameter_names  = True, 
    parameter_types  = (Real, Array), 
    parameter_ranks  = None, 
    parameter_shapes = None
)

#### Array Lookup

In [64]:
def array_lookup(environment, array, parameters):
    index = [int(p.first.value) for p in parameters]
    return array[tuple(index)]

built_ins.register_operation_function(
    operation        = 'binary', 
    name             = 'implicit_call', 
    function         = array_lookup, 
    parameter_names  = True, 
    parameter_types  = (Array, Array), 
    parameter_ranks  = None, 
    parameter_shapes = None
)

#### Conversion @, [implocit]

In [65]:
def unit_attachment(environment, tensor, units):
    if type(units) == Unit:
        units = Units(units)
    converted = units.convert_forwards(tensor)
    converted.first.unit = units
    return converted

def unit_change(environment, tensor, units):
    if type(units) == Unit:
        units = Units(units)
    if units == tensor.first.unit:
        tensor.first.unit = units
    else:
        raise Exception('Units not compatiple')
    
    return tensor

built_ins.register_operation_function(
    operation        = 'binary', 
    name             = 'implicit_unit', 
    function         = unit_attachment, 
    parameter_names  = True, 
    parameter_types  = (None, Unit), 
    parameter_ranks  = (1, 1), 
    parameter_shapes = None
)
built_ins.register_operation_function(
    operation        = 'binary', 
    name             = '@', 
    function         = unit_change, 
    parameter_names  = True, 
    parameter_types  = (None, Unit), 
    parameter_ranks  = None, 
    parameter_shapes = None
)

### Builtin functions

#### Absolute value: abs

In [66]:
def absolute(A):
    return Tensor(Real(abs(A.first.value)))

built_ins.register_function('abs', absolute, None, (Real,), (0,))

#### Round: round

In [67]:
def _round(A):
    return Tensor(Real( A.first.value.quantize(Decimal(1)) ))

built_ins.register_function('round', _round, None, (Real,), (0,))

#### Round down: floor

In [68]:
def floor(A):
    return Tensor(Real( A.first.value.quantize(Decimal(1), rounding=decimal.ROUND_DOWN) ))

built_ins.register_function('floor', floor, None, (Real,), (0,))

#### Round up: ceil

In [69]:
def ceil(A):
    return Tensor(Real( A.first.value.quantize(Decimal(1), rounding=decimal.ROUND_UP) ))

built_ins.register_function('ceil', ceil, None, (Real,), (0,))

#### Logarithm: log

In [70]:
def log(A, B):
    return Tensor(Real( math.log( A.first.value, B.first.value ) ))
    
built_ins.register_function('log', log, None, (Real,Real), (0,0))

### Conversion Specifiers

#### Distance

In [71]:
# def km_forwards(km):
#     return Real(km.value * Decimal(1000))

# def km_backwards(m):
#     return Real(m.value / Decimal(1000))

# def  m_forwards(m):
#     return Real(m.value)

# def  m_backwards(m):
#     return Real(m.value)

# def cm_forwards(cm):
#     return Real(cm.value / Decimal(100))

# def cm_backwards(m):
#     return Real(m.value * Decimal(100))

# def mm_forwards(mm):
#     return Real(mm.value / Decimal(1000))

# def mm_backwards(m):
#     return Real(m.value * Decimal(1000))

# def um_forwards(um):
#     return Real(um.value / Decimal(1000000))

# def um_backwards(m):
#     return Real(m.value * Decimal(1000000))

# def nm_forwards(nm):
#     return Real(nm.value / Decimal(1000000000))

# def nm_backwards(m):
#     return Real(m.value * Decimal(1000000000))

# def pm_forwards(pm):
#     return Real(pm.value / Decimal(1000000000000))

# def pm_backwards(m):
#     return Real(m.value * Decimal(1000000000000))


# def in_forwards(_in):
#     return Real(_in.value * Decimal('0.0254'))

# def in_backwards(m):
#     return Real(m.value / Decimal('0.0254'))

# def ft_forwards(ft):
#     return Real(ft.value * Decimal('0.3048'))

# def ft_backwards(m):
#     return Real(m.value / Decimal('0.3048'))

# def yd_forwards(yd):
#     return Real(yd.value * Decimal('0.9144'))

# def yd_backwards(m):
#     return Real(m.value / Decimal('0.9144'))

# def mi_forwards(mi):
#     return Real(mi.value * Decimal('1609.344'))

# def mi_backwards(m):
#     return Real(m.value / Decimal('1609.344'))

# built_ins.register_conversion_function_set('km', km_forwards, km_backwards)
# built_ins.register_conversion_function_set( 'm',  m_forwards,  m_backwards)
# built_ins.register_conversion_function_set('cm', cm_forwards, cm_backwards)
# built_ins.register_conversion_function_set('mm', mm_forwards, mm_backwards)
# built_ins.register_conversion_function_set('um', um_forwards, um_backwards)
# built_ins.register_conversion_function_set('nm', nm_forwards, nm_backwards)
# built_ins.register_conversion_function_set('pm', pm_forwards, pm_backwards)

# built_ins.register_conversion_function_set('in', in_forwards, in_backwards)
# built_ins.register_conversion_function_set('ft', ft_forwards, ft_backwards)
# built_ins.register_conversion_function_set('yd', yd_forwards, yd_backwards)
# built_ins.register_conversion_function_set('mi', mi_forwards, mi_backwards)

#### Area

In [72]:
# def km2_forwards(km):
#     return Real(km.value * Decimal(1000**2))

# def km2_backwards(m):
#     return Real(m.value / Decimal(1000**2))

# def  m2_forwards(m):
#     return Real(m.value)

# def  m2_backwards(m):
#     return Real(m.value)

# def cm2_forwards(cm):
#     return Real(cm.value / Decimal(100**2))

# def cm2_backwards(m):
#     return Real(m.value * Decimal(100**2))

# def mm2_forwards(mm):
#     return Real(mm.value / Decimal(1000**2))

# def mm2_backwards(m):
#     return Real(m.value * Decimal(1000**2))

# def um2_forwards(um):
#     return Real(um.value / Decimal(1000000**2))

# def um2_backwards(m):
#     return Real(m.value * Decimal(1000000**2))

# def nm2_forwards(nm):
#     return Real(nm.value / Decimal(1000000000**2))

# def nm2_backwards(m):
#     return Real(m.value * Decimal(1000000000**2))

# def pm2_forwards(pm):
#     return Real(pm.value / Decimal(1000000000000**2))

# def pm2_backwards(m):
#     return Real(m.value * Decimal(1000000000000**2))

# built_ins.register_conversion_function_set('km2', km2_forwards, km2_backwards)
# built_ins.register_conversion_function_set( 'm2',  m2_forwards,  m2_backwards)
# built_ins.register_conversion_function_set('cm2', cm2_forwards, cm2_backwards)
# built_ins.register_conversion_function_set('mm2', mm2_forwards, mm2_backwards)
# built_ins.register_conversion_function_set('um2', um2_forwards, um2_backwards)
# built_ins.register_conversion_function_set('nm2', nm2_forwards, nm2_backwards)
# built_ins.register_conversion_function_set('pm2', pm2_forwards, pm2_backwards)

#### Volume

In [73]:
# def km3_forwards(km):
#     return Real(km.value * Decimal(1000**3))

# def km3_backwards(m):
#     return Real(m.value / Decimal(1000**3))

# def  m3_forwards(m):
#     return Real(m.value)

# def  m3_backwards(m):
#     return Real(m.value)

# def   l_forwards(l):
#     return Real(l.value / Decimal(10**3))

# def   l_backwards(m):
#     return Real(m.value * Decimal(10**3))

# def cm3_forwards(cm):
#     return Real(cm.value / Decimal(100**3))

# def cm3_backwards(m):
#     return Real(m.value * Decimal(100**3))

# def mm3_forwards(mm):
#     return Real(mm.value / Decimal(1000**3))

# def mm3_backwards(m):
#     return Real(m.value * Decimal(1000**3))

# def um3_forwards(um):
#     return Real(um.value / Decimal(1000000**3))

# def um3_backwards(m):
#     return Real(m.value * Decimal(1000000**3))

# def nm3_forwards(nm):
#     return Real(nm.value / Decimal(1000000000**3))

# def nm3_backwards(m):
#     return Real(m.value * Decimal(1000000000**3))

# def pm3_forwards(pm):
#     return Real(pm.value / Decimal(1000000000000**3))

# def pm3_backwards(m):
#     return Real(m.value * Decimal(1000000000000**3))

# built_ins.register_conversion_function_set('km3', km3_forwards, km3_backwards)
# built_ins.register_conversion_function_set( 'm3',  m3_forwards,  m3_backwards)
# built_ins.register_conversion_function_set(  'l',   l_forwards,   l_backwards)
# built_ins.register_conversion_function_set('cm3', cm3_forwards, cm3_backwards)
# built_ins.register_conversion_function_set( 'ml', cm3_forwards, cm3_backwards)
# built_ins.register_conversion_function_set('mm3', mm3_forwards, mm3_backwards)
# built_ins.register_conversion_function_set('um3', um3_forwards, um3_backwards)
# built_ins.register_conversion_function_set('nm3', nm3_forwards, nm3_backwards)
# built_ins.register_conversion_function_set('pm3', pm3_forwards, pm3_backwards)

#### Time

In [74]:
# def  ms_forwards(ms):
#     return Real(ms.value / Decimal(1000))

# def  ms_backwards(s):
#     return Real(s.value * Decimal(1000))

# def   s_forwards(s):
#     return Real(s.value)

# def   s_backwards(s):
#     return Real(s.value)

# def min_forwards(min):
#     return Real(min.value * Decimal(60))

# def min_backwards(s):
#     return Real(s.value / Decimal(60))

# def  hr_forwards(hr):
#     return Real(hr.value * Decimal(3600))

# def  hr_backwards(s):
#     return Real(s.value / Decimal(3600))

# def day_forwards(day):
#     return Real(day.value * Decimal(86400))

# def day_backwards(s):
#     return Real(s.value / Decimal(86400))

# def time_forwards(string):
#     pattern = r'(?:(\d+)(?:d|D)\s*)?(\d{1,2})(?:h|H)?:(\d{1,2})(?:m|M)?(?::(\d{1,2}(?:\.\d+)?)(?:s|S)?)?'
#     match = re.fullmatch(pattern, string.value.strip())
#     if match:
#         d = match.group(1) or 0
#         h = match.group(2) or 0
#         m = match.group(3) or 0
#         s = match.group(4) or 0
#         return Tensor(Real( Decimal(d) * 86400 + Decimal(h) * 3600 + Decimal(m) * 60 + Decimal(s)))
#     else:
#         raise Exception(f'Invalid time formatting: "{string.value}"')

# def time_backwards(t):
#     t = t.value
#     d = t // Decimal(86400)
#     t -= d * Decimal(86400)
#     h = t // Decimal(3600)
#     t -= h * Decimal(3600)
#     m = t // Decimal(60)
#     t -= m * Decimal(60)
#     s = t // Decimal(1)
#     t -= s * Decimal(1)
#     ms = t // Decimal('0.001')
#     t -= ms * Decimal('0.001')
#     ps = t / Decimal('0.000001')
    
#     str_d = str(int(d))
#     str_h = str(int(h)).zfill(2)
#     str_m = str(int(m)).zfill(2)
#     str_s = str(int(s)).zfill(2)
#     str_ms = str(int(ms)).zfill(3)
#     str_ps = str(int(ps)).zfill(3)
    
#     str_days = '' if d == 0 else                    f'{str_d}d ' 
#     str_time =                                      f'{str_h}h:{str_m}m:{str_s}'
#     str_fraction_1 = '' if ms == 0 and ps == 0 else f'.{str_ms}'
#     str_fraction_2 = '' if ps == 0 else             f'{str_ps}'
    
#     return Tensor(String( str_days+str_time+str_fraction_1+str_fraction_2+'s' ))
        

# built_ins.register_conversion_function_set(  'ms',   ms_forwards,   ms_backwards)
# built_ins.register_conversion_function_set(   's',    s_forwards,    s_backwards)
# built_ins.register_conversion_function_set( 'min',  min_forwards,  min_backwards)
# built_ins.register_conversion_function_set(  'hr',   hr_forwards,   hr_backwards)
# built_ins.register_conversion_function_set( 'day',  day_forwards,  day_backwards)
# built_ins.register_conversion_function_set('time', time_forwards, time_backwards)

### Units

#### Distance

In [75]:
# distance = {
#     'km': {
#         'forwards':  Decimal('1000'),
#         'backwards': Decimal('.001'),
#     },
#     'm': {
#         'forwards':  Decimal('1'),
#         'backwards': Decimal('1'),
#     },
#     'dm': {
#         'forwards':  Decimal('.1'),
#         'backwards': Decimal('10'),
#     },
#     'cm': {
#         'forwards':  Decimal('.01'),
#         'backwards': Decimal('100'),
#     },
#     'mm': {
#         'forwards':  Decimal('.001'),
#         'backwards': Decimal('1000'),
#     },
#     'um': {
#         'forwards':  Decimal('.000001'),
#         'backwards': Decimal('1000000'),
#     },
#     'nm': {
#         'forwards':  Decimal('.000000001'),
#         'backwards': Decimal('1000000000'),
#     },
#     'pm': {
#         'forwards':  Decimal('.000000000001'),
#         'backwards': Decimal('1000000000000'),
#     },
#     'in': {
#         'forwards':  Decimal('0.0254'),
#         'backwards': Decimal('39.37007874015748031496062992'),
#     },
#     'tf': {
#         'forwards':  Decimal('0.3048'),
#         'backwards': Decimal('3.280839895013123359580052493'),
#     },
#     'yd': {
#         'forwards':  Decimal('0.9144'),
#         'backwards': Decimal('1.093613298337707786526684164'),
#     },
#     'mi': {
#         'forwards':  Decimal('1609.344'),
#         'backwards': Decimal('0.0006213711922373339696174341844'),
#     },
# }

# built_ins.register_unit_group('distance', distance)

#### Volume

In [76]:
# dm = Unit(
#     'distance', 
#     'dm', 
#     distance['dm']['forwards'], 
#     distance['dm']['backwards']
# )

# cm = Unit(
#     'distance', 
#     'cm', 
#     distance['cm']['forwards'], 
#     distance['cm']['backwards']
# )

# L  = Units({dm: 3}, 'L')
# mL = Units({cm: 3}, 'mL')
# cc = Units({cm: 3}, 'cc')

# built_ins.register_unit('L', L)
# built_ins.register_unit('mL', mL)
# built_ins.register_unit('cc', cc)

#### Time

In [77]:
# time = {
#     'd': {
#         'forwards':  Decimal('86400'),
#         'backwards': Decimal('.00001157407407407407407407407407'),
#     },
#     'hr': {
#         'forwards':  Decimal('3600'),
#         'backwards': Decimal('.0002777777777777777777777777778'),
#     },
#     'min': {
#         'forwards':  Decimal('60'),
#         'backwards': Decimal('.01666666666666666666666666667'),
#     },
#     's': {
#         'forwards':  Decimal('1'),
#         'backwards': Decimal('1'),
#     },
#     'ms': {
#         'forwards':  Decimal('.001'),
#         'backwards': Decimal('1000'),
#     },
#     'us': {
#         'forwards':  Decimal('.000001'),
#         'backwards': Decimal('1000000'),
#     },
#     'ns': {
#         'forwards':  Decimal('.000000001'),
#         'backwards': Decimal('1000000000'),
#     }
# }

# built_ins.register_unit_group('time', time)

#### Angles

In [78]:
# angle = {
#     'deg': {
#         'forwards':  Decimal('0.01745329251994329576923690768'),
#         'backwards': Decimal('57.29577951308232087679815483'),
#     },
#     'grad': {
#         'forwards':  Decimal('0.01570796326794896619231321692'),
#         'backwards': Decimal('63.66197723675813430755350533'),
#     },
#     'rad': {
#         'forwards':  Decimal('1'),
#         'backwards': Decimal('1'),
#     },
#     'rev': {
#         'forwards':  Decimal('6.283185307179586476925286767'),
#         'backwards': Decimal('0.1591549430918953357688837634'),
#     },
# }

# built_ins.register_unit_group('angle', angle)

#### Angular Speeds

In [79]:
# rev = Unit(
#     'angle', 
#     'rev', 
#     angle['rev']['forwards'], 
#     angle['rev']['backwards'],
# )

# _min = Unit(
#     'time',
#     'min',
#     time['min']['forwards'],
#     time['min']['backwards'],
# )

# rpm = Units({rev: 1, _min: -1}, 'rpm')

# built_ins.register_unit('rpm', rpm)

# Define Token

## Tokens Types

In [80]:
TOKEN_TYPE_STRING     = 'string'
TOKEN_TYPE_INTEGER    = 'integer'
TOKEN_TYPE_NUMBER     = 'number'
TOKEN_TYPE_OPERATOR   = 'operand'
TOKEN_TYPE_OPEN_GROUP = 'group'
TOKEN_TYPE_LITERAL    = 'literal'

TOKEN_TYPE_OPEN_GROUP        = 'open group'
TOKEN_TYPE_CLOSE_GROUP       = 'close group'
TOKEN_TYPE_NEW_ITEM          = 'new item'
TOKEN_TYPE_BINARY            = 'binary operand'
TOKEN_TYPE_BINARY_CONTINUING = 'continuing binary operand'
TOKEN_TYPE_UNARY_LEFT        = 'left unary operand'
TOKEN_TYPE_UNARY_RIGHT       = 'right unary operand'

OPERAND_TYPES = [
    TOKEN_TYPE_OPEN_GROUP,
    TOKEN_TYPE_CLOSE_GROUP, 
    TOKEN_TYPE_NEW_ITEM, 
    TOKEN_TYPE_BINARY, 
    TOKEN_TYPE_BINARY_CONTINUING, 
    TOKEN_TYPE_UNARY_LEFT, 
    TOKEN_TYPE_UNARY_RIGHT, 
]

TOKEN_TYPE_VALUE = [
    TOKEN_TYPE_STRING,
    TOKEN_TYPE_INTEGER,
    TOKEN_TYPE_NUMBER,
    TOKEN_TYPE_LITERAL,
]

## Token Operations

In [81]:
token_definitions = {
    TOKEN_TYPE_OPEN_GROUP:        built_ins.groups,
    TOKEN_TYPE_CLOSE_GROUP:       built_ins.groups, 
    TOKEN_TYPE_NEW_ITEM:          built_ins.new_items, 
    TOKEN_TYPE_BINARY:            built_ins.binary_operators, 
    TOKEN_TYPE_BINARY_CONTINUING: built_ins.binary_operators_continuing, 
    TOKEN_TYPE_UNARY_LEFT:        built_ins.left_unary_operators, 
    TOKEN_TYPE_UNARY_RIGHT:       built_ins.right_unary_operators, 
}

## Operation Indexing

In [82]:
def find_token_definition(string, token_type, f = lambda x:x.token):
    for token_definition in token_definitions[token_type]:
        if string == f(token_definition):
#         if re.fullmatch(operator.re_match, string) != None:
            return token_definition
    return None

## Define Token Regular Expressions

In [83]:
def re_join(l, f=lambda x:x):
    items = [f(i) for i in l]
    items.sort(key=lambda x:len(x), reverse=True)
    return '(' + ')|('.join([re.escape(i) for i in items]) + ')'

# regex
re_number =    r"""((([\.][0-9]+)|([0-9]+[\.]?[0-9]*))([eE][-+]?[0-9]+)?)"""
re_integer =   r"""((0b|0o|0d|0x|[0-9]+_)[0-9a-zA-Z,]+)"""
re_string =    r"""((\""".*?\""")|('''.*?''')|(".*?")|('.*?'))"""
re_literal =   r"""([A-Za-z_][A-Za-z0-9_]*)"""


re_open_group =                 re_join(built_ins.groups, lambda x:x.open_token)
re_close_group =                re_join(built_ins.groups, lambda x:x.close_token)
re_binary_operands =            re_join(built_ins.binary_operators, lambda x:x.token)
re_binary_continuing_operands = re_join(built_ins.binary_operators_continuing, lambda x:x.token)
re_left_unary_operands =        re_join(built_ins.left_unary_operators, lambda x:x.token)
re_right_unary_operands =       re_join(built_ins.right_unary_operators, lambda x:x.token)
re_new_item =                   re_join(built_ins.new_items, lambda x:x.token)


re_tokens = {
    TOKEN_TYPE_STRING:            re_string,
    TOKEN_TYPE_INTEGER:           re_integer,
    TOKEN_TYPE_NUMBER:            re_number,
    TOKEN_TYPE_LITERAL:           re_literal,
    TOKEN_TYPE_OPEN_GROUP:        re_open_group,
    TOKEN_TYPE_BINARY:            re_binary_operands,
    TOKEN_TYPE_BINARY_CONTINUING: re_binary_continuing_operands,
    TOKEN_TYPE_UNARY_LEFT:        re_left_unary_operands,
    TOKEN_TYPE_UNARY_RIGHT:       re_right_unary_operands,
    TOKEN_TYPE_NEW_ITEM:          re_new_item,
    TOKEN_TYPE_CLOSE_GROUP:       re_close_group,
}

re_tokens

{'string': '((\\""".*?\\""")|(\'\'\'.*?\'\'\')|(".*?")|(\'.*?\'))',
 'integer': '((0b|0o|0d|0x|[0-9]+_)[0-9a-zA-Z,]+)',
 'number': '((([\\.][0-9]+)|([0-9]+[\\.]?[0-9]*))([eE][-+]?[0-9]+)?)',
 'literal': '([A-Za-z_][A-Za-z0-9_]*)',
 'open group': '(\\|\\|)|(\\()|(\\[)|(\\{)|(\\|)',
 'binary operand': '(implicit_unit)|(implicit_call)|(matmul)|(mod)|(dot)|(\\/\\/)|(\\.\\*)|(\\=\\>)|(\\+)|(\\-)|(\\*)|(4)|(\\/)|(\\^)|(\\#)|(\\$)|(\\?)|(\\=)|(\\@)',
 'continuing binary operand': '(matmul)|(mod)|(dot)|(\\/\\/)|(\\.\\*)|(\\+)|(\\-)|(\\*)|(\\/)|(\\^)|(\\#)|(\\@)',
 'left unary operand': '(log10)|(log2)|(asin)|(acos)|(atan)|(sin)|(cos)|(tan)|(ln)|(\\+)|(\\-)|(\\=)|(\\$)',
 'right unary operand': '(\\!)',
 'new item': '(\\,)|(\\;)|(\\\n)',
 'close group': '(\\|\\|)|(\\))|(\\])|(\\})|(\\|)'}

# Parse

## Lexing

In [84]:
def re_match_length(string, re_pattern):
    match = re.match(re_pattern, string)
    return match.span()[1] if match != None else 0

In [85]:
class Lexer:
    def __init__(self, ans_available=False):
        self.tokens = []
        self.ans_available = ans_available
        
        self.source_string = ''
        self.current_ln_offset = 0
        self.ln_count = 0
        self.source = Source()
        
    def process(self, string):
        # TOKEN_TYPE_STRING
        # TOKEN_TYPE_INTEGER
        # TOKEN_TYPE_NUMBER
        # TOKEN_TYPE_LITERAL

        # TOKEN_TYPE_OPEN_GROUP
        # TOKEN_TYPE_BINARY
        # TOKEN_TYPE_BINARY_CONTINUING
        # TOKEN_TYPE_UNARY_LEFT
        # TOKEN_TYPE_UNARY_RIGHT
        # TOKEN_TYPE_NEW_ITEM
        # TOKEN_TYPE_CLOSE_GROUP
        
        tokens = []
        
        i = len(self.source_string)
        string = self.source_string = self.source_string + string
        self.source.set(string)
        
        while i < len(string):

            if string[i] in ' ':
                i += 1
                continue
                
            offset = i - self.current_ln_offset
            ln = self.ln_count
            
            if string[i] == '\n':
                self.current_ln_offset = i+1
                self.ln_count += 1

            if len(tokens) > 0:
                last_token_type =  tokens[-1].token_type
            elif len(self.tokens) > 0:
                last_token_type = self.tokens[-1].token_type
            else:
                last_token_type = None
                
            allowed_token_types = []
            allowed_token_types_with_implicit_op = {}
            allowed_token_types_with_ans = {}

            # begin with
            if last_token_type in [
                None,
            ]:

                allowed_token_types = [
                    TOKEN_TYPE_UNARY_LEFT,
                    TOKEN_TYPE_OPEN_GROUP,
                    TOKEN_TYPE_NEW_ITEM,
                    TOKEN_TYPE_CLOSE_GROUP,
                    *TOKEN_TYPE_VALUE,
                ]
                if self.ans_available:
                    allowed_token_types_with_ans = {
                        TOKEN_TYPE_UNARY_RIGHT: '',
                        TOKEN_TYPE_BINARY_CONTINUING: '',
                    }

            if last_token_type in [\
                TOKEN_TYPE_OPEN_GROUP,
                TOKEN_TYPE_NEW_ITEM,
            ]:
                allowed_token_types = [
                    TOKEN_TYPE_UNARY_LEFT,
                    TOKEN_TYPE_OPEN_GROUP,
                    TOKEN_TYPE_NEW_ITEM,
                    TOKEN_TYPE_CLOSE_GROUP,
                    *TOKEN_TYPE_VALUE,
                ]

            # after value excluding literal
            if last_token_type in [
                TOKEN_TYPE_CLOSE_GROUP,
                TOKEN_TYPE_UNARY_RIGHT,
                TOKEN_TYPE_STRING,
                TOKEN_TYPE_INTEGER,
                TOKEN_TYPE_NUMBER,
            ]:
                allowed_token_types = [
                    TOKEN_TYPE_UNARY_RIGHT,
                    TOKEN_TYPE_BINARY,
                    TOKEN_TYPE_NEW_ITEM,
                    TOKEN_TYPE_CLOSE_GROUP,
                ]
                allowed_token_types_with_implicit_op = {
                    TOKEN_TYPE_OPEN_GROUP: 'implicit_unit',
                    TOKEN_TYPE_LITERAL: 'implicit_unit',
                }

            # after literal value
            if last_token_type in [
                TOKEN_TYPE_LITERAL,
            ]:
                allowed_token_types = [
                    TOKEN_TYPE_UNARY_RIGHT,
                    TOKEN_TYPE_BINARY,
                    TOKEN_TYPE_NEW_ITEM,
                    TOKEN_TYPE_CLOSE_GROUP,
                ]
                allowed_token_types_with_implicit_op = {
                    TOKEN_TYPE_OPEN_GROUP: 'implicit_call',
                    TOKEN_TYPE_LITERAL: 'implicit_call',
                }

            # after operator
            if last_token_type in [
                TOKEN_TYPE_BINARY,
                TOKEN_TYPE_BINARY_CONTINUING,
                TOKEN_TYPE_UNARY_LEFT,

            ]:
                allowed_token_types = [
                    TOKEN_TYPE_OPEN_GROUP,
                    TOKEN_TYPE_UNARY_LEFT,
                    *TOKEN_TYPE_VALUE,
                ]


            # find matching token type
            token_type, token_str = None, None

            all_allowed_token_types = [
                list(allowed_token_types_with_ans.items()),
                [(t, None) for t in allowed_token_types],
                list(allowed_token_types_with_implicit_op.items()),
            ]
            all_allowed_token_types = sum(all_allowed_token_types, [])

            for possible_token_type, implicit_op in all_allowed_token_types:
                re_pattern = re_tokens[possible_token_type]

                l = re_match_length(string[i:], re_pattern)
                if l > 0:

                    if implicit_op == '':
                        tokens.append(Token(TOKEN_TYPE_LITERAL, implicit_op, ln, offset, self.source))
                    elif implicit_op != None:
                        tokens.append(Token(TOKEN_TYPE_BINARY, implicit_op, ln, offset, self.source))

                    token_type = possible_token_type
                    token_str = string[i:i+l]
                    break



            # invalid token
            if token_type == None:
                raise TokenNotAllowedException(self.source, ln, offset)
                i += 1
            else:
                tokens.append(Token(token_type, token_str, ln, offset, self.source))
                i += len(token_str)

        self.tokens += tokens
        return tokens  

## Treeify

In [86]:
def bubble_up(focus, new):
    """
    Finds ancestor/parent in tree upwards from `token` thats the first group token or the first
    """
    
    while True:
        if type(focus) == NodeGroup or focus.precedence < new.precedence:
            return focus
        else:
            focus = focus.parent

In [87]:
def bubble_up_to_group(focus):
    while True:
        if type(focus) == NodeGroup:
            return focus
        else:
            focus = focus.parent

In [88]:
class TokenTreeBuilder:
    
    def __init__(self):
        self.root = NodeGroup(find_token_definition('{', TOKEN_TYPE_OPEN_GROUP, lambda x:x.open_token), None)
        self.focus = self.root

    def build(self, tokens):
    
        # TOKEN_TYPE_STRING
        # TOKEN_TYPE_INTEGER
        # TOKEN_TYPE_NUMBER
        # TOKEN_TYPE_LITERAL

        # TOKEN_TYPE_OPEN_GROUP
        # TOKEN_TYPE_BINARY
        # TOKEN_TYPE_UNARY_LEFT
        # TOKEN_TYPE_UNARY_RIGHT

        # TOKEN_TYPE_NEW_ITEM
        # TOKEN_TYPE_CLOSE_GROUP

        for token in tokens:


            # focus is
            # Binary Operator
            # Left Unary Operator
            if type(self.focus) in [
                NodeBinary,
                NodeUnaryLeft,
            ]:

                # next is 
                # Unary left
                if token.token_type == TOKEN_TYPE_UNARY_LEFT:

                    operand = find_token_definition(token.string, TOKEN_TYPE_UNARY_LEFT)
                    next_node = NodeUnaryLeft(operand)

                    self.focus.set_right(next_node)
                    self.focus = next_node

                # next is
                # Group
                elif token.token_type == TOKEN_TYPE_OPEN_GROUP:

                    group = find_token_definition(token.string, TOKEN_TYPE_OPEN_GROUP, lambda x:x.open_token)
                    next_node = NodeGroup(group)
                    self.focus.set_right(next_node)
                    self.focus = next_node

                # next is
                # TOKEN_TYPE_STRING
                # TOKEN_TYPE_INTEGER
                # TOKEN_TYPE_NUMBER
                # TOKEN_TYPE_LITERAL
                elif token.token_type in TOKEN_TYPE_VALUE:

                    next_node = NodeValue(token.string, token.token_type)
                    self.focus.set_right(next_node)
                    self.focus = next_node

                else:
                    raise TokenNotAllowedException(token)


            # focus is
            # Value
            # Closed Group
            elif type(self.focus) == NodeValue or (type(self.focus) == NodeGroup and self.focus.is_complete()):

                # next is
                # Binary
                if token.token_type in [TOKEN_TYPE_BINARY, TOKEN_TYPE_BINARY_CONTINUING]:

                    operand = find_token_definition(token.string, TOKEN_TYPE_BINARY)
                    next_node = NodeBinary(operand)

                    parent_node = bubble_up(self.focus.parent, next_node)
                    child_node = parent_node.get_right()
                    parent_node.set_right(next_node)
                    next_node.set_left(child_node)
                    self.focus = next_node


                # next is
                # Unary right
                elif token.token_type == TOKEN_TYPE_UNARY_RIGHT:

                    operand = find_token_definition(token.string, TOKEN_TYPE_UNARY_RIGHT)
                    next_node = NodeUnaryRight(operand)

                    parent_node = self.focus.parent # bubble_up(self.focus.parent, next_node)
                    child_node = parent_node.get_right()
                    parent_node.set_right(next_node)
                    next_node.set_left(child_node)
                    # self.focus = next_node

                # next is
                # New item
                elif token.token_type == TOKEN_TYPE_NEW_ITEM:

                    token_definition = find_token_definition(token.string, TOKEN_TYPE_NEW_ITEM)

                    parent_node = bubble_up_to_group(self.focus.parent)

                    if token_definition.token in parent_node.group_definition.seperators:

                        depth = parent_node.group_definition.seperators[token_definition.token]
                        if depth > 0:
                            parent_node.increase(depth)
                            self.focus = parent_node
                    else:
                        raise TokenNotAllowedException(token)


                # next is
                # Close group
                elif token.token_type == TOKEN_TYPE_CLOSE_GROUP:

                    parent_node = bubble_up_to_group(self.focus.parent)
                    parent_node.close()
                    self.focus = parent_node

                # next is
                else:
                    raise TokenNotAllowedException(token)


            # focus is
            # Open Group
            elif type(self.focus) == NodeGroup and not self.focus.is_complete(): 

                # next is
                # Binary - use ans on left
                if token.token_type == TOKEN_TYPE_BINARY:

                    left = NodeValue('ans', TOKEN_TYPE_LITERAL)

                    operand = find_token_definition(token.string, TOKEN_TYPE_BINARY)
                    next_node = NodeBinary(operand)
                    next_node.set_left(left)

                    self.focus.add(next_node)
                    self.focus = next_node

                # next is
                # Unary left
                elif token.token_type == TOKEN_TYPE_UNARY_LEFT:

                    operand = find_token_definition(token.string, TOKEN_TYPE_UNARY_LEFT)
                    next_node = NodeUnaryLeft(operand)

                    self.focus.add(next_node)
                    self.focus = next_node

                # next is
                # Group
                elif token.token_type == TOKEN_TYPE_OPEN_GROUP:

                    group = find_token_definition(token.string, TOKEN_TYPE_OPEN_GROUP, lambda x:x.open_token)
                    next_node = NodeGroup(group)
                    self.focus.add(next_node)
                    self.focus = next_node

                # next is
                # Close Group
                elif token.token_type == TOKEN_TYPE_CLOSE_GROUP:

                    self.focus.close()

                # next is
                # TOKEN_TYPE_STRING
                # TOKEN_TYPE_INTEGER
                # TOKEN_TYPE_NUMBER
                # TOKEN_TYPE_LITERAL
                elif token.token_type in TOKEN_TYPE_VALUE:

                    next_node = NodeValue(token.string, token.token_type)
                    self.focus.add(next_node)
                    self.focus = next_node

                # next is 
                # TOKEN_TYPE_NEW_ITEM
                elif token.token_type == TOKEN_TYPE_NEW_ITEM:

                    token_definition = find_token_definition(token.string, TOKEN_TYPE_NEW_ITEM)

                    if token_definition.token in self.focus.group_definition.seperators:

                        depth = self.focus.group_definition.seperators[token_definition.token]
                        if depth > 0:
                            self.focus.increase(depth)
                    else:
                        raise TokenNotAllowedException(token)

                # next is
                else:
                    raise TokenNotAllowedException(token)



        return self.root


## Computation Graph

In [89]:
def build_computation_graph(token_tree):
    return token_tree.get_evaluable()

# Excecute

## Environment

In [90]:
class Environment():
       
    def __init__(self, parent=None, dictionary=None):
        self.parent = parent
        self.dictionary = dictionary or {}
        self.ln_count = 1
        self.ans_available = False
        
    def __getitem__(self, key):
        
        if key in self.dictionary:
            return self.dictionary[key]
        elif self.parent != None:
            return self.parent[key]
        else:
            assert False, f'key {key} is not defined in this scope'
        
    def __setitem__(self, key, value):
        
        if type(key) in [list, tuple] and type(value) in [list, tuple]:
            for k,v in zip(key, value):
                self[key] = value
        
        else:
            if key in self.dictionary:
                self.dictionary[key] = value
            elif self.parent != None:
                self.parent[key] = value
            else:
                self.dictionary[key] = value
    
    def __contains__(self, key):
        
        if key in self.dictionary:
            return True
        elif self.parent != None:
            return key in self.parent
        else:
            return False
        
    def getLnCount(self):
        return str(self.ln_count)
        
    def addLn(self, value, code, environment):
        in_name = f'in{self.ln_count}'
        out_name = f'out{self.ln_count}'
        self[in_name] = define_function(environment, (), code)
        self[out_name] = value
        self[''] = value
        
        self.ln_count += 1
        self.ans_available = True
        
        return out_name
    
    def enterScope(self, dictionary=None):
        return Environment(self, dictionary)
        
    def exitScope(self):
        return self if self.parent == None else self.parent
        
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        return f'Environment({self.dictionary, self.parent})'

In [91]:
def create_base_environment():
    built_in_variables = {k:v for k,v in built_ins.vars}
    built_in_functions = {}
    built_in_conversion_function_sets = {}
    built_in_units = {}
    
    for name, function, parameter_names, parameter_types, parameter_ranks, parameter_shapes, output_unit in built_ins.functions:
        
        parameter_evaluation_parameters = built_ins.function_parameter_evaluation_parameters.get(name, {})
        
        built_in_functions[name] = built_in_functions.get(name, FunctionSet(name, parameter_evaluation_parameters))
        built_in_functions[name].add(FunctionSignature(
            name, 
            function, 
            parameter_names, 
            parameter_types, 
            parameter_ranks, 
            parameter_shapes,
            output_unit
        ))
        
    for name, forward, reverse in built_ins.conversion_function_sets:
        built_in_conversion_function_sets[name] = ConversionFunctionSet(name, forward, reverse)
        
    for name, unit in built_ins.units:
        built_in_units[f'<unit>{name}'] = unit
    
    environment = Environment(None, {**built_in_variables, 
                                     **built_in_functions, 
                                     **built_in_conversion_function_sets, 
                                     **built_in_units
                                    })
    
    return environment

## Evaluate

In [92]:
def evaluate(computation_graph, environment):
    return computation_graph.eval(environment)

In [93]:
def read(string, environment, lexer=None, token_tree_builder=None, **kwargs):
    
    lexer = lexer or Lexer(environment.ans_available)
    tokens = lexer.process(string)
    
    token_tree_builder = token_tree_builder or TokenTreeBuilder()
    token_tree = token_tree_builder.build(tokens)
    is_complete = token_tree.is_complete(True)
    
    return {
        'environment': environment,
        'lexer':       lexer,
        'token_tree_builder': token_tree_builder,
        'is_complete': is_complete,
    }

In [94]:
def execute(token_tree_builder, environment, **kwargs):
    computation_graph = build_computation_graph(token_tree_builder.root)
    result = evaluate(computation_graph, environment)
    ln_name = environment.addLn(result, computation_graph, environment)
    print_result(result, ln_name)

    return result

In [95]:
def print_result(result, ln_name):
    ln_name = ln_name + ': '
    space = ' ' * len(ln_name)
    lines = str(result).split('\n')
    pre = [ln_name] + (len(lines)-1) * [space]
    
    lines = [p+l for p,l in zip(pre, lines)]
    
    print(*lines, sep='\n')

## Run

In [96]:
def calc(query, environment, debug=False):

    context = read(query, environment)
    result = execute(**context)
    return result

## Test

### Basics

In [97]:
e = create_base_environment()

In [98]:
calc('0.1 + 0.2', e)

out1: 0.3


Tensor([Real(0.3, None)])

In [99]:
calc('[phi, 1/phi]', e)

out2: [1.61803398874989484820458683436563811772, 0.6180339887498948482045868344]


Tensor([Real(1.61803398874989484820458683436563811772, None)
 Real(0.6180339887498948482045868344, None)], (2,))

In [100]:
calc('(1 + [1,2,3]) * 2; 1 + [1,2,3] * 2', e)

out3: [3, 5, 7]


Tensor([Real(3, None) Real(5, None) Real(7, None)], (3,))

In [101]:
calc('2 ^ [1,2,3,4]; [1,2,3,4] ^ 2', e)

out4: [1, 4, 9, 16]


Tensor([Real(1, None) Real(4, None) Real(9, None) Real(16, None)], (4,))

In [102]:
calc('[2,4,8] ^ [2,3,4]', e)

out5: [4, 64, 4096]


Tensor([Real(4, None) Real(64, None) Real(4096, None)], (3,))

In [103]:
calc('PI * [1/2, 1, 2,,,]', e)

out6: [[[1.570796326794896619231321692, 3.141592653589793238462643383, 6.283185307179586476925286767]]]


Tensor([Real(1.570796326794896619231321692, None)
 Real(3.141592653589793238462643383, None)
 Real(6.283185307179586476925286767, None)], (1, 1, 3))

In [104]:
calc('[1,2,,3,4,,5,6] # [1,2,3,,4,5,6]', e)

out7: [[9, 12, 15],
       [19, 26, 33],
       [29, 40, 51]]


Tensor([Real(9, None) Real(12, None) Real(15, None) Real(19, None) Real(26, None)
 Real(33, None) Real(29, None) Real(40, None) Real(51, None)], (3, 3))

In [105]:
calc('|0 - |10-100| |; |0-1|; ||0-1||; |[1,-1,2,-2]|', e)

out8: [1, 1, 2, 2]


Tensor([Real(1, None) Real(1, None) Real(2, None) Real(2, None)], (4,))

In [106]:
calc('$[1,4,9,16]; 3$[1,8,27,64]; 0.1$10; 10^(1/0.1)', e)

out9: 10000000000


Tensor([Real(10000000000, None)])

In [107]:
calc('[1,2,3] .* [3,2,1]; [1,2,3,,] # [3,,2,,1]; [1,2,3,,4,5,6] .* [4,5,6,,7,8,9]', e)

out10: [32, 122]


Tensor([Real(32, None) Real(122, None)], (2,))

In [108]:
calc("""
[
    1,2,3,,
    4,5,6
    ,,,
    6,5,4,,
    3,2,1
    ,,,
    1,2,3,,
    4,5,6
]
""", e)

out11: [[[1, 2, 3],
         [4, 5, 6]],
       
        [[6, 5, 4],
         [3, 2, 1]],
       
        [[1, 2, 3],
         [4, 5, 6]]]


Tensor([Real(1, None) Real(2, None) Real(3, None) Real(4, None) Real(5, None)
 Real(6, None) Real(6, None) Real(5, None) Real(4, None) Real(3, None)
 Real(2, None) Real(1, None) Real(1, None) Real(2, None) Real(3, None)
 Real(4, None) Real(5, None) Real(6, None)], (3, 2, 3))

In [109]:
calc("""
[[[1, 2, 3],
  [4, 5, 6]],

 [[6, 5, 4],
  [3, 2, 1]],

 [[1, 2, 3],
  [4, 5, 6]]]
""", e)

out12: [[[1, 2, 3],
         [4, 5, 6]],
       
        [[6, 5, 4],
         [3, 2, 1]],
       
        [[1, 2, 3],
         [4, 5, 6]]]


Tensor([Real(1, None) Real(2, None) Real(3, None) Real(4, None) Real(5, None)
 Real(6, None) Real(6, None) Real(5, None) Real(4, None) Real(3, None)
 Real(2, None) Real(1, None) Real(1, None) Real(2, None) Real(3, None)
 Real(4, None) Real(5, None) Real(6, None)], (3, 2, 3))

### Variables

In [110]:
calc('x = 1; y = 2; x = 3; x^y', e)

out13: 9


Tensor([Real(9, None)])

In [111]:
calc('a = ((2, 4), 8); ((x, y), z) = a; x^y; y^z', e)

out14: 65536


Tensor([Real(65536, None)])

In [112]:
calc('x = [1,2,,3,4]; (a,b,c,d) = x; (a,b,c,d)', e)

out15: (1, 2, 3, 4)


Array([Tensor([Real(1, None)]) Tensor([Real(2, None)]) Tensor([Real(3, None)])
 Tensor([Real(4, None)])])

### Implicit Multiplication

In [113]:
# print(calc('x = 5; 2 * x; 2(3)(3+4)x; 3x^2'))

In [114]:
# print(calc('x = 5; 2 x x'))

In [115]:
# print(calc('f = x => 2x; 3 f(5)'))

### Functions

In [116]:
calc('f = x => x^3; f(5)', e)

out16: 125


Tensor([Real(125, None)])

In [117]:
calc('f = (x, y) => x^y; f(3,4); f([1,2,3], [4,5,6])', e)

out17: [1, 32, 729]


Tensor([Real(1, None) Real(32, None) Real(729, None)], (3,))

In [118]:
calc('f = x => {y = x + 1; x^y}; f(3)', e)

out18: 81


Tensor([Real(81, None)])

In [119]:
calc("""
    f = x => {
        inc = x => x + 1
        y = inc(x)
        x^y
    }
    g = x => x^x
    
    (
        f(2)
        g(2)
    )
""", e)

out19: (8, 4)


Array([Tensor([Real(8, None)]) Tensor([Real(4, None)])])

In [120]:
calc('f = () => 1; f()', e)

out20: 1


Tensor([Real(1, None)])

In [121]:
calc("""
f = x => (x, x^2, x^3)
(a,b,c) = f(3)
[c,b,a]
""", e)

out21: [27, 9, 3]


Tensor([Real(27, None) Real(9, None) Real(3, None)], (3,))

### Array and Tensor Item Lookup

In [122]:
calc("""
    f = x => x^2
    c = 1
    a = (1,2,3)
    t = [0.1, 0.2, 0.3,, 0.4, 0.5, 0.6]
    t(1,c)
    a(c)
""", e)

out22: 2


Tensor([Real(2, None)])

In [123]:
calc('a = (1,2,3); a(1)', e)

out23: 2


Tensor([Real(2, None)])

In [124]:
calc('t = [1,2,3]; t(1)', e)

out24: 2


Tensor([Real(2, None)])

### Conversions

In [125]:
calc('120cm', e)

out25: 120.00cm


Tensor([Real(1.20, cm)])

In [126]:
calc('120cm + 15cm', e)

Units -> Units cm None
out26: 135.00cm


Tensor([Real(1.35, cm)])

In [127]:
calc('120cm + 15mm', e)

Units -> Units cm None
out27: 121.500cm


Tensor([Real(1.215, cm)])

In [128]:
calc('20m / 5s', e)

Units -> Units m/s None
out28: 4m/s


Tensor([Real(4, m/s)])

In [129]:
calc('20m * 10cm', e)

Units -> Units m^2 None
out29: 2.00m^2


Tensor([Real(2.00, m^2)])

In [130]:
calc('[20(m^3/s^2), 5(m/s*m), 5(m*s^-1)]', e)

out30: ^3/s^2, 5m^2/s, 5m/s]


Tensor([Real(20, m^3/s^2) Real(5, m^2/s) Real(5, m/s)], (3,))

In [131]:
calc('(1hr + 35min) * 3', e)

Units -> Units hr None
Units -> Units hr None
out31: 4.750000000000000000000000000hr


Tensor([Real(17100, hr)])

In [132]:
calc('[5m ^ 2, 4$16s]', e)

Units -> Units m None
Units -> Units s None
out32: , 2.000000000000000000000000000s]


Tensor([Real(25, m) Real(2.000000000000000000000000000, s)], (2,))

In [133]:
calc('1m * 1cm @ (mm^2)', e)

Units -> Units m^2 None
out33: 10000.00mm^2


Tensor([Real(0.01, mm^2)])

In [134]:
calc('[1,2,3]cm', e)

out34: 1.00cm


Tensor([Real(0.01, cm)])

In [135]:
calc('[1mm, 1cm, 1m, 1km] @ m', e)

out35: [0.001m, 1.00cm, 1m, 1.000km]


Tensor([Real(0.001, m) Real(0.01, cm) Real(1, m) Real(1000, km)], (4,))

In [136]:
calc('160cm * 50cm * 40cm @ L', e)

Units -> Units cm^2 None
Units -> Units cm^3 None
out36: 320.000000L


Tensor([Real(0.320000, L)])

In [137]:
calc('2L / 50(cm^2) @ (L/m^2)', e)

Units -> Units dm None
out37: 4.0dm


Tensor([Real(0.4, dm)])

In [138]:
calc('1m / 20cm', e)

out38: 5


Tensor([Real(5, None)])

In [139]:
calc('1L / 1dm', e)

Units -> Units dm^2 None
out39: 1.00dm^2


Tensor([Real(0.01, dm^2)])

In [140]:
calc('1000rpm @ rev/s', e)

out40: 16.66666666666666666666666667rev/s


Tensor([Real(104.7197551196597746154214461, rev/s)])

In [141]:
calc('[tan 10deg, atan 10/10 @ deg]', e)

Units -> Units rad None
out41: [0.17632698070846497540031805328908376395702362060546875, 44.99999999999999824582267538deg]


Tensor([Real(0.17632698070846497540031805328908376395702362060546875, None)
 Real(0.78539816339744827899949086713604629039764404296875, deg)], (2,))

In [142]:
calc('10cm / 100000000(m/s)', e)

Units -> Units s None
out42: 1E-9s


Tensor([Real(1E-9, s)])

In [143]:
# t = calc('1/2L', e)

# t.first.unit

In [144]:
# calc('1hr + 5min + 1000ms @ time', e)

In [145]:
# calc("'1d 2:40:0.0001'time + '3d 1:35:0.001'time @ time", e)

In [146]:
# calc("'1:35'time * 3 @ time", e)

In [147]:
# calc("('10:00'time * 5) mod 24hr @ time", e)

In [148]:
# calc('"1h:2m:3s"time * 30 @ time', e)

In [149]:
calc('x = 5cm @ mm; x', e)

out43: 50.00mm


Tensor([Real(0.05, mm)])

### Continued Equations

In [150]:
# calc('3 ^ 4', e)
# calc(' / 4', e)
execute(**read('3 ^ 4', e))
execute(**read(' / 4', e))

out44: 81
out45: 20.25


Tensor([Real(20.25, None)])

In [151]:
calc('2 ^ 3', e)
calc('out37 * 2', e)

out46: 8
Units -> Units dm None
out47: 8.0dm


Tensor([Real(0.8, dm)])

In [152]:
# calc('[1,2', e)
context = read('[1,2', e)
context = read(',3]', **context)
execute(**context)

out48: [1, 2, 3]


Tensor([Real(1, None) Real(2, None) Real(3, None)], (3,))

In [153]:
# calc('1 +', e)
context = read('1 +', e)
context = read(' 2', **context)
execute(**context)

out49: 3


Tensor([Real(3, None)])

In [154]:
context = read('[1,2,,'   , e)
context = read(' 3,4,,,'  , **context)
context = read(''         , **context)
context = read(' 5,6,,'   , **context)
context = read(' 7,8]'    , **context)
execute(**context)

out50: [[[1, 2],
         [3, 4]],
       
        [[5, 6],
         [7, 8]]]


Tensor([Real(1, None) Real(2, None) Real(3, None) Real(4, None) Real(5, None)
 Real(6, None) Real(7, None) Real(8, None)], (2, 2, 2))

In [155]:
context = read('    f = x => {  ', e)
context = read('\n    y = 2 * x ', **context)
context = read('\n    x * y     ', **context)
context = read('\n  }; f(3)     ', **context)
execute(**context)
context['lexer'].source

out51: 18


In [156]:
context = read("""
    f = x => {
        inc = x => x + 1
        y = inc(x)
        x^y
    }
    g = x => x^x
    
    (
        f(2)
        g(2)
    )
""", e)
execute(**context)
print(context['lexer'].tokens[40].getSourcePointerString())

out52: (8, 4)
10:         f(2)
            ↑


### Post Variable Assignment

In [157]:
calc('x = 5', e)
calc('3 ^ 4', e)
calc('= x', e)
calc('x', e)

out53: 5
out54: 81
out55: 81
out56: 81


Tensor([Real(81, None)])

### Syntaxt error pointers

In [158]:
# calc('123 ** 3', e)

In [159]:
# calc('(1,2;3)', e)

In [160]:
# calc('123 / 0', e)

In [161]:
# calc('non_existing_var', e)

### Reuse Code Snipits

In [162]:
e = create_base_environment()
calc('x = 2', e)
calc('x * 3', e)
calc('x = 5', e)
calc('in2()', e)
calc('out2', e)

out1: 2
out2: 6
out3: 5
out4: 15
out5: 6


Tensor([Real(6, None)])

### Functions

In [163]:
e = create_base_environment()
calc('-3 ? abs', e)
calc('abs(-3)', e)
calc('|-3|', e)

out1: 3
out2: 3
out3: 3


Tensor([Real(3, None)])

In [164]:
calc('round(1.73)', e)
calc('1.73 ? round', e)
calc('1.23 ? round', e)
calc('1.23 ? floor', e)
calc('1.23 ? ceil', e)

out4: 2
out5: 2
out6: 1
out7: 1
out8: 2


Tensor([Real(2, None)])

In [165]:
calc('100!', e)
calc('0!', e)
calc('1!', e)
calc('2!', e)
calc('3!', e)

out9: 9.332621544394415268169923877E+157
out10: 1
out11: 1
out12: 2
out13: 6


Tensor([Real(6, None)])